****This notebook provide the covid-19 predictions for around 60 countries about the number of active infected persons
---

At the beginning of the COVID 2020 epidemic, many people were discussing the factors that could affect the disease such as smoking, age for example. I would like to know wether those are the part of the important factors. I was also interested to use China epidemic values as reference to model the evaluation of the epidemic in the other countries. I believe that China can really be taken as a reference as its policy (quarantine) was consistent throughout the epidemic (from beginning to end) without considering the economic loss.

The project will address two questions

The effect of external factors (tests, temperature, health and other descriptors of the population, policy: quarantine, school closing, restrictions) on the epidemic
Model the evolution of the epidemic value based on China numbers as reference

For the **factor analysis**, I'm using the decision tree model to analyze which factors are the most important. For the random forest, I'm plotting the feature importance.For the KNeighbors am also plotting the decision boundaries for each class. I compare all models to the "most-frequent" baseline to know how much information are captured by them. I'm using those models because they can be easily interpreted.
​
The input data will be the external factors and the target are the total number of infected and the deaths ratio. To make this a classification problem, I'm creating 6 categories from low to high and explore two manners to achieve this
​
- Equidistant bins
- "Quantile bins" to have a balanced target
​
Since the number of data points is low, I always report the cross-validated mean test score on the entire data set and only split the data into train/test sets for reporting purpose with the classification matrix.
​
For the **epidemic modelling**, I'm fitting the values from the China epidemic to other countries. To achieve this, I'm finding the epidemic peak and duration from start to peak, and fitting this to the other countries peak and duration using RMSE as a metric. To model the evolution of the numbers for each countrie, I'm also fitting a polynomial to the current points.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

corona daten analyze and predictions
---

At the beginning of the COVID 2020 epidemic, many people were discussing the factors that could affect the disease such as smoking, age for example. I would like to know wether those are the part of the important factors. I was also interested to use China epidemic values as reference to model the evaluation of the epidemic in the other countries. I believe that China can really be taken as a reference as its policy (quarantine) was consistent throughout the epidemic (from beginning to end) without considering the economic loss.

The project will address two questions

The effect of external factors (tests, temperature, health and other descriptors of the population, policy: quarantine, school closing, restrictions) on the epidemic
Model the evolution of the epidemic value based on China numbers as reference
This was a very iterative project where the workflow evolved a lot after each finding and results.

the data mostly will be get from internet to be actual

data source : https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
              https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports


Read actual Data 
---

In [ ]:
import requests
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
r = requests.get(url, allow_redirects=True)
open('./time_series_covid19_confirmed_US.csv', 'wb').write(r.content)
#
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
r = requests.get(url, allow_redirects=True)
open('./time_series_covid19_confirmed_global.csv', 'wb').write(r.content)
#
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
r = requests.get(url, allow_redirects=True)
open('./time_series_covid19_deaths_US.csv', 'wb').write(r.content)
#
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
r = requests.get(url, allow_redirects=True)
open('./time_series_covid19_deaths_global.csv', 'wb').write(r.content)
#
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
r = requests.get(url, allow_redirects=True)
open('./time_series_covid19_recovered_global.csv', 'wb').write(r.content)
#
from datetime import datetime, timedelta
url=datetime.strftime(datetime.today() - timedelta(1), 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/%m-%d-%Y.csv')
r = requests.get(url, allow_redirects=True)
open('./csse_covid_19_daily_reports.csv', 'wb').write(r.content)


`covid19_by_country.csv`
---

Source: [COVID-19 Predictors](https://www.kaggle.com/nightranger77/covid19-demographic-predictors?select=covid19_by_country.csv) version 19 (12.mai.2020)

Gives meta-information about policies and how the epidemic is handled in the different countries

Note that COVID-19 testing data will not be updated; however, COVID-19 infections and deaths from the Johns Hopkins dataset will be updated every few days.

Combines the Johns Hopkins COVID-19 data with several other public datasets

2018 GDP in Million USD in covid https://data.worldbank.org/indicator/NY.GDP.MKTP.CD

Crime and Population https://worldpopulationreview.com/countries/crime-rate-by-country/

Smoking rate https://ourworldindata.org/smoking#prevalence-of-smoking-across-the-world

Sex (% Female) https://data.worldbank.org/indicator/SP.POP.TOTL.FE.ZS

Median Age https://worldpopulationreview.com/countries/median-age/

Also includes COVID-19 specific data from @koryto https://www.kaggle.com/koryto/countryinfo

Health expenditure (% of GDP) : https://data.worldbank.org/indicator/SH.XPD.CHEX.GD.ZS

from https://www.kaggle.com/koryto/countryinfo

```
pop             : Population in 1000
tests           : The number of corona detection tests made per day (as for 3/24/2020)
testpop         : pop / tests (as for 3/24/2020)
density         : The amount of citizens per square meter
medianage       : Median age
urbanpop        : The % of people who lives in urban locations.
quarantine      : Initial date of quarantine policy
schools         : The initial date of school closure policy
publicplace     : The initial date of public places (restaurents, bars, etc.) restrictions
hospibed        : The amount of hospital bed per 1K people
smokers         : The % of smokers within the population.
sex0            : Amount of males per female at birth (2020)
sex14           : Amount of males per female in the age group 0-14
sex25           : Amount of males per female in the age group 14-25
sex54           : Amount of males per female in the age group 26-54
sex64           : Amount of males per female in the age group 55-64
sex65plus       : Amount of males per female in the age group 65+
sexratio        : Amount of males per female (not separated by age)
lungDeath       : rate from lung diseases per 100k people
femalelungDeath : rate from lung diseases per 100k people for female
malelungDeath   : rate from lung diseases per 100k people for male
gdp2019Nominal  : GDP for 2019 (in 1 million USD)
spme explanations
```

In [ ]:
import shutil

original = r'../input/input1/covid19_by_country.csv'
target = r'./covid19_by_country.csv'
shutil.copyfile(original, target)
original = r'../input/input1/GlobalLandTemperaturesByCountry.csv'
target = r'./GlobalLandTemperaturesByCountry.csv'
shutil.copyfile(original, target)
original = r'../input/input1/GlobalLandTemperaturesByMajorCity.csv'
target = r'./GlobalLandTemperaturesByMajorCity.csv'
shutil.copyfile(original, target)
original = r'../input/input1/API_SH.XPD.CHEX.GD.ZS_DS2_en_csv_v2_989101.csv'
target = r'./API_SH.XPD.CHEX.GD.ZS_DS2_en_csv_v2_989101.csv'
shutil.copyfile(original, target)
original = r'../input/input1/shift.JPG'
target = r'./shift.JPG'
shutil.copyfile(original, target)

print(os.listdir("./"))


Section 0: Data Preperation

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# Loading datasets required for analysis

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", color_codes=True)
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")

# Input data files are available in the "../input/" directory.
import os
print(os.listdir("./"))
#print(os.listdir("../input/input"))
%load_ext autoreload
%autoreload 2
import plotly.io as pio
pio.renderers.default = "browser"

data cleaning

In [ ]:
codiv_country=pd.read_csv('./covid19_by_country.csv')
codiv_country.sample(10)

In [ ]:
codiv_country.describe()

In [ ]:
#nomalize some datas, also change the population in real size
codiv_country['Population 2020']=codiv_country['Population 2020']*1000
codiv_country['Tests_per_10kp']=codiv_country['Tests']*10000.0/codiv_country['Population 2020']
codiv_country['Tests_per_10kp_log1p']=np.log1p(codiv_country['Tests_per_10kp'])
codiv_country['GDP 2018 per_1p_log1p']=np.log1p((codiv_country['GDP 2018']/codiv_country['Population 2020']))

Some are skewed, apply the log-transform in such cases NB: Density is difficult to use ie canada , usa, russia have high density in big city and 0 outside, epidemy is mostly in big city
I will drop all `sex` and `Crime Index` which are not part of the analysis. I will also drop the `Total` columns since I collect this information from the real time data set `csse_covid_19_daily_reports.csv`

Testpop = Population/test, this is less interesing, better would be test*10000/population = number of test by 10000 person/day
Also the drop of density can be explain that the density in belgium is higher then in china, this is on country level, so not interesting

Female  = % female, this is the invers von sex ratio, we will drop sex ratio

In [ ]:
codiv_country=codiv_country.drop([
    'Sex Ratio','Crime Index','Density','Tests_per_10kp','Test Pop','sex0','sex14','sex25','sex54','sex64','sex65plus','Total Infected','Total Deaths','Total Recovered','Tests','GDP 2018','Population 2020'
], axis=1)
codiv_country.sample(10)

Addind a new information : Health expenditure/GDP in %
---
Health expenditure (% of GDP) : https://data.worldbank.org/indicator/SH.XPD.CHEX.GD.ZS
the file dont have informations for 2018-2019, so i will use the 2017 data

In [ ]:
Health_expenditure=pd.read_csv(os.path.join('./', 'API_SH.XPD.CHEX.GD.ZS_DS2_en_csv_v2_989101.csv'))
Health_expenditure=Health_expenditure.set_index('Country Name')
Health_expenditure.sample(3)

In [ ]:
codiv_country['Health expenditure Ratio'] = 0.0
for countryindex in Health_expenditure.index:
    # If the country exists in the other table
    if not codiv_country[codiv_country['Country']==countryindex].empty :
        codiv_country.at[codiv_country['Country']==countryindex,'Health expenditure Ratio']=Health_expenditure[Health_expenditure.index==countryindex]['2017'][0]


Checking the distribution of the variables

In [ ]:
codiv_country.hist(figsize=(12, 12))
plt.show()

Check data types

In [ ]:
codiv_country.dtypes

Handling data types
---
We have information for 95 countries and 26 attributes left

Missing values
---
Checking for missing values. It seems that 15 countries have decided to put in place a quarantine and 67 not. But we should be careful about how quarantine is defined. For instance, the quarantine in France is not the same as in Wuhan.

In [ ]:
codiv_country.isna().sum()

In [ ]:
codiv_country[codiv_country['Health expenditure Ratio'].isnull()]

codiv_country[codiv_country['Health expenditure Ratio'].isnull()]

In [ ]:
codiv_country['Health expenditure Ratio'] = codiv_country['Health expenditure Ratio'].fillna(codiv_country['Health expenditure Ratio'].mean())

If the number of Tests_per_10kp is missing, I will assume that the country is not testing and I will replace by zero.

In [ ]:
codiv_country['Tests_per_10kp_log1p'] = codiv_country['Tests_per_10kp_log1p'].fillna(0) 

Later, I will join the tables together, and hence need to make sure they have the same country names.

In [ ]:
codiv_country.Country[codiv_country.Country == "United States"] = "US"

checking outlier by some columns
---

In [ ]:
z_scores_Females= (codiv_country['Females 2018']- codiv_country['Females 2018'].mean()) /codiv_country['Females 2018'].std()
z_scores_Females.plot()
plt.hlines(-1,0,100,colors="red")
plt.hlines(-2,0,100,colors="yellow")
plt.hlines(-3,0,100,colors="green")
plt.hlines(1,0,100,colors="red")
plt.hlines(2,0,100,colors="yellow")
plt.hlines(3,0,100,colors="green")
plt.ylabel("z_scores_Females")
plt.xlabel("country index")

In [ ]:
codiv_country['Females 2018'].plot()
plt.ylabel("Females %")
plt.xlabel("country index")

In [ ]:
idx_Females = (np.abs(z_scores_Females) > 2)
codiv_country['Females 2018'][idx_Females]=codiv_country['Females 2018'].mean()
print('z-scores_Females:', z_scores_Females.shape)
codiv_country['Females 2018'].plot()

In [ ]:
z_scores_Female_Lung= (codiv_country['Female Lung']- codiv_country['Female Lung'].mean()) /codiv_country['Female Lung'].std()
z_scores_Female_Lung.plot()
plt.hlines(-1,0,100,colors="red")
plt.hlines(-2,0,100,colors="yellow")
plt.hlines(-3,0,100,colors="green")
plt.hlines(1,0,100,colors="red")
plt.hlines(2,0,100,colors="yellow")
plt.hlines(3,0,100,colors="green")
plt.ylabel("z_scores_Female_Lung")
plt.xlabel("country index")

In [ ]:
idx_Female_Lung = (np.abs(z_scores_Female_Lung) > 2)
codiv_country['Female Lung'][idx_Female_Lung]=codiv_country['Female Lung'].mean()
print('z-scores_Female_Lung:', z_scores_Female_Lung.shape)

In [ ]:
z_scores_Male_Lung= (codiv_country['Male Lung']- codiv_country['Male Lung'].mean()) /codiv_country['Male Lung'].std()
z_scores_Male_Lung.plot()
plt.hlines(-1,0,100,colors="red")
plt.hlines(-2,0,100,colors="yellow")
plt.hlines(-3,0,100,colors="green")
plt.hlines(1,0,100,colors="red")
plt.hlines(2,0,100,colors="yellow")
plt.hlines(3,0,100,colors="green")
plt.ylabel("z_scores_Male_Lung")
plt.xlabel("country index")

In [ ]:
idx_Male_Lung = (np.abs(z_scores_Male_Lung) > 2)
codiv_country['Male Lung'][idx_Male_Lung]=codiv_country['Male Lung'].mean()
print('z-scores_Male_Lung:', z_scores_Male_Lung.shape)

In [ ]:
z_scores_Lung= (codiv_country['lung']- codiv_country['lung'].mean()) /codiv_country['lung'].std()
z_scores_Lung.plot()
plt.hlines(-1,0,100,colors="red",label="np.abs(z_scores_Females) > 1)")
plt.hlines(-2,0,100,colors="yellow",label="np.abs(z_scores_Females) > 2)")
plt.hlines(-3,0,100,colors="green",label="np.abs(z_scores_Females) > 3)")
plt.hlines(1,0,100,colors="red",label="np.abs(z_scores_Females) > 1)")
plt.hlines(2,0,100,colors="yellow",label="np.abs(z_scores_Females) > 2)")
plt.hlines(3,0,100,colors="green",label="np.abs(z_scores_Females) > 3)")
plt.ylabel("z_scores_Lung")
plt.xlabel("country index")

In [ ]:
idx_Lung = (np.abs(z_scores_Lung) > 2)
codiv_country['lung'][idx_Lung]=codiv_country['lung'].mean()
print('z-scores_Lung:', z_scores_Lung.shape)

Remark: 

1. The informations like number of bed reflect the political decision : 
        ie in belgium aged poeple in hospice have no right for hospital, so the number of needed bed is not huge
        ie, in wuhan the was building extra new hospital to give youg and aged poeple a chance to get healty
thats why i will not considere parameter depending of the political impact

2. The number of test, there is missing the time information about the test number, ie, if the test capacity increased, then when

working on the time serie file csse_covid_19_daily_reports.csv
---
Removing columns that I won't use in the analysis

In [ ]:
codiv_csse=pd.read_csv(os.path.join('./', 'csse_covid_19_daily_reports.csv'))
codiv_csse=codiv_csse.drop(['FIPS','Admin2','Last_Update','Combined_Key','Long_','Lat'], axis=1)
codiv_csse.sample(10)

The number of "comfirmed" is referred to in the first DataFrame as "Infected". I will rename it

In [ ]:
codiv_csse = codiv_csse.rename(columns={'Confirmed': 'Infected'})

Now, I'm aggregating the information by country. Two countries will be affected: US and China.

In [ ]:
codiv_csse=codiv_csse.groupby('Country_Region').sum().reset_index()
codiv_csse.sample(10)

New feature : deaths_ratio
---
                create deaths_ratio = number of deaths*1000 / number of infected


deaths_ratio means how much from 1000 infected persons was dying

In [ ]:
codiv_csse["Deaths Ratio"]=codiv_csse["Deaths"]*1000/codiv_csse["Infected"]
codiv_csse.sample(10)

Later, I will only analyze the top 60 countries where the epidemic is the worse.

In [ ]:
set(codiv_country['Country'].unique()) - set(codiv_csse['Country_Region'].unique())

Because of the joining that I will do later, I also need to normalize the country names.

In [ ]:
codiv_csse.Country_Region[codiv_csse.Country_Region == "Korea, South"] = "South Korea"
codiv_csse.Country_Region[codiv_csse.Country_Region == "Czechia"] = "Czech Republic"

Setting the index to Countries

In [ ]:
codiv_csse=codiv_csse.set_index('Country_Region')
codiv_csse.sample(10)

In [ ]:
codiv_csse.describe()

Checking data type

In [ ]:
codiv_csse.dtypes

In [ ]:
codiv_csse.isna().sum()

Create the total infected, deaths, recovered and infected columns for the `codiv_country` DataFrame. I however have to check that the country is in the `codiv_csse` DataFrame which will not be used anymore later in the analysis. I will create this column with the following fomula

$$\text{Total Active} = \text{Confirmed} - \text{Deaths} - \text{Recovered}$$

In [ ]:
codiv_country['Total Infected'] = 0.0
codiv_country['Total Deaths'] = 0.0
codiv_country['Total Recovered'] = 0.0
codiv_country['Total Active'] = 0.0
codiv_country['Deaths Ratio'] = 0.0
for countryindex in codiv_csse.index:
    # If the country exists in the other table
    if not codiv_country[codiv_country['Country']==countryindex].empty :
        codiv_country.at[codiv_country['Country']==countryindex,'Total Infected Log10']=np.log1p(codiv_csse[codiv_csse.index==countryindex]['Infected'][0])
        codiv_country.at[codiv_country['Country']==countryindex,'Total Infected']=codiv_csse[codiv_csse.index==countryindex]['Infected'][0]
        codiv_country.at[codiv_country['Country']==countryindex,'Total Deaths Log10']=np.log1p(codiv_csse[codiv_csse.index==countryindex]['Deaths'][0])
        codiv_country.at[codiv_country['Country']==countryindex,'Total Deaths']=codiv_csse[codiv_csse.index==countryindex]['Deaths'][0]
        codiv_country.at[codiv_country['Country']==countryindex,'Total Recovered Log10']=np.log1p(codiv_csse[codiv_csse.index==countryindex]['Recovered'][0])
        codiv_country.at[codiv_country['Country']==countryindex,'Total Recovered']=codiv_csse[codiv_csse.index==countryindex]['Recovered'][0]
        codiv_country.at[codiv_country['Country']==countryindex,'Deaths Ratio']=codiv_csse[codiv_csse.index==countryindex]['Deaths Ratio'][0]
        if codiv_csse[codiv_csse.index==countryindex]['Active'][0] ==0.0:
            codiv_country.at[codiv_country['Country']==countryindex,'Total Active Log10']=np.log1p(codiv_csse[codiv_csse.index==countryindex]['Infected'][0]-codiv_csse[codiv_csse.index==countryindex]['Deaths'][0]-codiv_csse[codiv_csse.index==countryindex]['Recovered'][0])
            codiv_country.at[codiv_country['Country']==countryindex,'Total Active']=codiv_csse[codiv_csse.index==countryindex]['Infected'][0]-codiv_csse[codiv_csse.index==countryindex]['Deaths'][0]-codiv_csse[codiv_csse.index==countryindex]['Recovered'][0]
        else:
            codiv_country.at[codiv_country['Country']==countryindex,'Total Active Log10']=np.log1p(codiv_csse[codiv_csse.index==countryindex]['Active'][0])
            codiv_country.at[codiv_country['Country']==countryindex,'Total Active']=codiv_csse[codiv_csse.index==countryindex]['Active'][0]

In [ ]:
codiv_country.isnull()[['Total Infected', 'Total Deaths', 'Total Recovered', 'Total Active', 'Deaths Ratio']].sum()

For the second and 3rd part i will build new table, covid without restriction / with quarantine / with rectriction 

In [ ]:
codiv_country_qurantine=codiv_country[codiv_country['Quarantine'].notnull()]
codiv_country_Restrictions=codiv_country[codiv_country['Restrictions'].notnull()]
codiv_country_without_Restrictions_qurantine=codiv_country[codiv_country['Restrictions'].isnull() & codiv_country['Quarantine'].isnull()]

print("codiv_country_qurantine shape" , codiv_country_qurantine.shape)
print("codiv_country_Restrictionse shape" , codiv_country_Restrictions.shape)
print("codiv_country_without_Restrictions_quarantine" , codiv_country_without_Restrictions_qurantine.shape)

Preparing the time series data
---
I'm dropping the Latitude and Longitude information which are not part of the analysis.

In [ ]:
codiv_time_confirmed=pd.read_csv(os.path.join('./', 'time_series_covid19_confirmed_global.csv'))
codiv_time_confirmed=codiv_time_confirmed.drop(['Lat','Long'], axis=1)

codiv_time_deaths=pd.read_csv(os.path.join('./', 'time_series_covid19_deaths_global.csv'))
codiv_time_deaths=codiv_time_deaths.drop(['Lat','Long'], axis=1)

codiv_time_recovered=pd.read_csv(os.path.join('./', 'time_series_covid19_recovered_global.csv'))
codiv_time_recovered=codiv_time_recovered.drop(['Lat','Long'], axis=1)

codiv_time_confirmed.sample(10)

---
codiv_country_temp and codiv_country_tempMajorCity
---
The temperature table deliver 1 value/month from 1700 till 2013, 
- the epidemic started in january, till april,
- The actual temperature value are not available, so i till build the data based on the last 3 years available to protect against temperature fluctuations


In [ ]:
codiv_country_temp=pd.read_csv('.//GlobalLandTemperaturesByCountry.csv')
#codiv_country_temp.loc['2020-01-01':'2020-04-01']
codiv_country_temp=codiv_country_temp.drop(['AverageTemperatureUncertainty'], axis=1)
codiv_country_temp=codiv_country_temp[((codiv_country_temp['dt'] > '2013-01-01') & (codiv_country_temp['dt'] < '2013-04-01')) | 
                                      ((codiv_country_temp['dt'] > '2012-01-01') & (codiv_country_temp['dt'] < '2012-04-01')) | 
                                      ((codiv_country_temp['dt'] > '2011-01-01') & (codiv_country_temp['dt'] < '2011-04-01'))]
codiv_country_temp = codiv_country_temp.groupby(['Country'])['AverageTemperature'].mean()
print(codiv_country_temp.sample(10))
print("codiv_country_temp",codiv_country_temp.shape)

In [ ]:
print(codiv_country_temp.describe())

The temperature table deliver 1 value/month till 2013, so i use the last 3 Years periode jan to april and make the mean.

the temperature is mainly done from big city information. epidemie is mostly also in big cities

In [ ]:
codiv_country_tempMajorCity=pd.read_csv('.//GlobalLandTemperaturesByMajorCity.csv')

codiv_country_tempMajorCity=codiv_country_tempMajorCity.drop(['AverageTemperatureUncertainty','Latitude','Longitude'], axis=1)
codiv_country_tempMajorCity=codiv_country_tempMajorCity[((codiv_country_tempMajorCity['dt'] > '2013-01-01') & (codiv_country_tempMajorCity['dt'] < '2013-04-01')) | 
                                      ((codiv_country_tempMajorCity['dt'] > '2012-01-01') & (codiv_country_tempMajorCity['dt'] < '2012-04-01')) | 
                                      ((codiv_country_tempMajorCity['dt'] > '2011-01-01') & (codiv_country_tempMajorCity['dt'] < '2011-04-01'))]
codiv_country_tempMajorCity= codiv_country_tempMajorCity.groupby(['Country'])['AverageTemperature'].mean()
print(codiv_country_tempMajorCity.sample(10))
print("codiv_country_tempMajorCity",codiv_country_tempMajorCity.shape)
print("243-49=194 countries dont have temperature for big city")

The epidemy happen mostly in big city
big country like usa/china/Russia have lot of unmanned area which have completely different temperature
That means, the representative temp, should be the one of the big cities ( if they exists)

In [ ]:
Temperature_difference=(codiv_country_tempMajorCity-codiv_country_temp).dropna()
Temperature_difference=Temperature_difference.sort_values( ascending=False)
x=Temperature_difference.index
y=Temperature_difference
fig, ax = plt.subplots(figsize=(20,10))
ax.scatter(x, y, alpha=0.5)
plt.xticks(rotation=45)
plt.ylabel("Delta Temp AvrMaincity-AvrCountry")
plt.grid(True)
plt.show()

you can see the adjustment for some country like russia and canada, the avr temp big city is slightly different for the avr country temperature, the same for the hot countries

complete the tabelle first  with the main city temp if they exists

In [ ]:
codiv_country['Temp_mean_jan_apr'] = 0.0
for countryindex in codiv_country_tempMajorCity.index:
    if not codiv_country[codiv_country['Country']==countryindex].empty :
        codiv_country.at[codiv_country['Country']==countryindex,'Temp_mean_jan_apr']=codiv_country_tempMajorCity.loc[countryindex]
codiv_country['Temp_mean_jan_apr'].sample(10)

complete missing value with mean country value

In [ ]:
for countryindex in codiv_country_temp.index:
    if not codiv_country[codiv_country['Country']==countryindex].empty:
        if codiv_country[codiv_country['Country']==countryindex]['Temp_mean_jan_apr'].values[0] == 0:
            codiv_country.at[codiv_country['Country']==countryindex,'Temp_mean_jan_apr']=codiv_country_temp.loc[countryindex]
codiv_country.head()

Checking for missing values

In [ ]:
codiv_country[(codiv_country['Temp_mean_jan_apr'] == 0)]

Two countries have missing values for the temperatures. I will keep them in the data and see if this produces issues later in the analysis.

The table have a heatmap per collumn, the higher value have dark red, smaller light red

now let show the data with total infected ascending=false
FIRST for the Total Infected

In [ ]:
transit=codiv_country.sort_values(by='Total Infected', ascending=False)
transit = transit.reset_index(drop=True)
transit.head(11).style.background_gradient(cmap='Blues')

let show the data with total active ascending=false for the active

In [ ]:
transit=codiv_country.sort_values(by='Total Active', ascending=False)
transit = transit.reset_index(drop=True)
transit.head(11).style.background_gradient(cmap='Greens')

Finaly let show the data with total Deaths ascending=false for the Deaths

In [ ]:
transit=codiv_country.sort_values(by='Total Deaths', ascending=False)
transit = transit.reset_index(drop=True)
transit.head(11).style.background_gradient(cmap='Reds')

In [ ]:
transit=codiv_country.sort_values(by='Deaths Ratio', ascending=False)
transit = transit.reset_index(drop=True)
transit.head(11).style.background_gradient(cmap='Oranges')

The deaths ratio deliver complete new tops. I think it is in relation to the land registration policy, ie how they register a codiv death
---
---
Section 1: data analyze
---
the interest is to check the worst country, so i limit the number  to 60 country  sorted on the higest infected

In [ ]:
codiv_country_short=codiv_country.sort_values(by='Total Infected', ascending=False).head(n=60).reset_index(drop=True)
codiv_country_short

In [ ]:
codiv_country_short.describe()

---
scatter plot for total infected
---
Checking the relation between Tests_per_10kp and total infected.

In [ ]:
sns.FacetGrid(codiv_country_short, hue="Country", size=6.5) \
   .map(plt.scatter, "Tests_per_10kp_log1p", "Total Infected Log10") \
   .add_legend()

plt.grid(True)

In [ ]:
#histogram of "total infected" for countries that have Tests_per_10kp_log1p == 0
codiv_country_short[codiv_country_short["Tests_per_10kp_log1p"] == 0]["Total Infected Log10"].hist(figsize=(6, 6), alpha=0.5, density=True)
plt.xlabel('Total Infected Log1p')
plt.ylabel('Nr Country density')
plt.title('Histogram Tests_per_10kp_log1p==0')

In [ ]:
#histogram of "total infected" for countries that have Tests_per_10kp_log1p != 0
codiv_country_short[codiv_country_short["Tests_per_10kp_log1p"] != 0]["Total Infected Log10"].hist(figsize=(6,6), alpha=0.5, density=True)
plt.xlabel('Total Infected Log1p')
plt.ylabel('Nr Country density')
plt.title('Histogram Tests_per_10kp_log1p not 0')

---
scatter plot for total active
---

In [ ]:
sns.FacetGrid(codiv_country_short, hue="Country", size=6.5) \
   .map(plt.scatter, "Tests_per_10kp_log1p", "Total Active Log10") \
   .add_legend()
plt.grid(True)

Historgram of "total infected" for countries that have Tests_per_10kp_log1p  == 0

In [ ]:
codiv_country_short[codiv_country_short["Tests_per_10kp_log1p"] == 0]["Total Active Log10"].hist(figsize=(6, 6), alpha=0.5, density=True)
plt.xlabel('Total Active Log10')
plt.ylabel('Nr Country density')
plt.title('Histogram Tests_per_10kp_log1p==0')

the country density and incted porsone is slightly higher then the country making test

histogram of "Total Active" for countries that have Tests_per_10kp_per_100ps_log1p > 0

In [ ]:
codiv_country_short[codiv_country_short["Tests_per_10kp_log1p"] != 0]["Total Active Log10"].hist(figsize=(6,6), alpha=0.5, density=True)
plt.xlabel('Total Active Log10')
plt.ylabel('Nr Country density')
plt.title('Histogram Tests_per_10kp_log1p not 0')

In [ ]:
codiv_country_short.drop([   'Total Infected', 'Total Deaths', 'Total Recovered', 'Total Active'], axis=1)

---
check some correlation possibilities
---
the goal is to check which parapemer could have an impact, the GDP/population size//density are take out, hmore population you have more it you will have infected., more poeple you have higher is the GDP, etc..
I will also check the pearsons correlation factor.

In [ ]:
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr
#I use a temp table to hide some columns not needed for the sns.
codiv_country_short_tmp=codiv_country_short.drop([ 'Deaths Ratio','Total Infected', 'Total Deaths', 'Total Recovered', 'Total Active','Total Recovered Log10','Total Infected Log10','Total Deaths Log10','Total Active Log10'], axis=1)
for col in codiv_country_short_tmp.loc[:, codiv_country_short_tmp.dtypes == np.number].keys():
    sns.jointplot(codiv_country_short['Total Infected Log10'], col, data=codiv_country_short, height=6, s=1, color="blue")
    corr, _ = pearsonr(codiv_country_short['Total Infected Log10'], codiv_country_short[col])
    print("Pearsons correlation Total Infected Log10 <-> %s : %.3f" %(col, corr))
    sns.jointplot(codiv_country_short['Total Deaths Log10'], col, data=codiv_country_short, height=6, s=1, color="red")
    corr, _ = pearsonr(codiv_country_short['Total Deaths Log10'], codiv_country_short[col])
    print("Pearsons correlation Total Deaths Log10 <-> %s : %.3f" %(col, corr))
    sns.jointplot(codiv_country_short['Total Active Log10'], col, data=codiv_country_short, height=6, s=1, color="green")
    corr, _ = pearsonr(codiv_country_short['Total Active Log10'], codiv_country_short[col])
    print("Pearsons correlation Total Active Log10 <-> %s : %.3f" %(col, corr))
    sns.jointplot(codiv_country_short['Deaths Ratio'], col, data=codiv_country_short, height=6, s=1, color="orange")
    corr, _ = pearsonr(codiv_country_short['Deaths Ratio'], codiv_country_short[col])
    print("Pearsons correlation Deaths Ratio <-> %s : %.3f" %(col, corr))
plt.show()

```
a low correlation Pearsons correlation Total Deaths Log10 <-> Health expenditure Ratio : 0.580 very strange
```
no correlation on number of smoking/male/female/lung (-0.5<Pearsons correlation<0.5), but look like an information on the avg temperature country having number of infect >4.0 (log) have an avr temp 2.5-7c over this temperature  the number of infected is lower., in cold temperature the same. the death show the same

There seems to be a correlation between "Health Expenditure Ratio" and Deaths/Infected which is hard to explain. Maybe it comes from other variables that are not part of the data. (confounding factors)

There is a peak of "total active" for mean temperature around 5°C. This similarity with worse countries cannot be explained by the latitude as seen below

- USA : 39,91N
- China : 39,55N
- Germany: 55N
- Italy:44N
- Spain 40N

In [ ]:
coefs_poly3 = np.polyfit(codiv_country_short["Temp_mean_jan_apr"], codiv_country_short["Total Infected Log10"], deg=8) 
sns.FacetGrid(codiv_country_short, hue="Country", size=6.5) \
   .map(plt.scatter, "Temp_mean_jan_apr", "Total Infected Log10") \
   .add_legend()
for indexq in range(-80,300,1):
    plt.plot(indexq/10,np.polyval(coefs_poly3, indexq/10), '.', color='black')
plt.vlines(7,7,14,colors="red",linestyles='dashed')
plt.vlines(24,7,14,colors="red",linestyles='dashed')
plt.grid(True)

It look like having 2 temperature peaks by 7c and 24c

In [ ]:
m,b = np.polyfit(codiv_country_short["Health expenditure Ratio"], codiv_country_short["Total Infected Log10"], 1) 

sns.FacetGrid(codiv_country_short, hue="Country", size=6.5) \
   .map(plt.scatter, "Health expenditure Ratio", "Total Infected Log10") \
   .add_legend()
plt.plot(codiv_country_short["Health expenditure Ratio"], m*codiv_country_short["Health expenditure Ratio"]+b, '--k') 
plt.grid(True)

More you invest in health more infected and death you have? or less you invest in health, less poeple are going to hospital and so less poeple are officialy registrated as infected?

China gave covid  test and medicine for free to everyone in the country, rich and poor. What about USA/europa?  if you have symptome please stay at home... that means they are not registrated as infected, if you are seriously sick, come over for a test (germany , belgium, switzerland and you have to pay for.)

---
---
Section 2: Analyzing if there are any external factors that affect the epidemic value
---

In this part of the analysis, I will fit, tune and evaluate a Decision Tree model, Forest model, and dummyclassifier to analyze the effect of external factors on the epidemic value.

In [ ]:
# Working on a copy
codiv_country_analyze = codiv_country.copy()

# Creating categorical variables
codiv_country_analyze['Quarantine_cat'] = codiv_country_analyze['Quarantine'].notnull().astype(int)
codiv_country_analyze['Restrictions_cat'] = codiv_country_analyze['Restrictions'].notnull().astype(int)
codiv_country_analyze['Schools_cat'] = codiv_country_analyze['Schools'].notnull().astype(int)

# 
codiv_country_analyze=codiv_country_analyze.drop([
    'Quarantine','Schools','Restrictions', # now categorical
    'Country', # not helpful
    
    # Only keep "Total Infected Log10"
    'Total Deaths','Total Infected','Total Active','Total Recovered', 
    "Total Deaths Log10","Total Recovered Log10","Total Active Log10","Deaths Ratio"
], axis=1)

codiv_country_analyze.sample(10)

In [ ]:
codiv_country_analyze[codiv_country_analyze.isnull().any(axis=1)]

In [ ]:
codiv_country_analyze = codiv_country_analyze[codiv_country_analyze.notnull().all(axis=1)]

Later, I will evaluate the decision tree and random forest estimators. I will store the results in a DataFrame.

In [ ]:
columns = ['method','Cross-validation']
index=range(8)
df_resultat = pd.DataFrame(index=index, columns=columns)

Create equidistant and balanced categories for the all 4 models
---

i think there is 2 different approche from the categories : 
- equidistant category
- category with same number of elements (_balanced)
we have a log10 number tables, classical outliers are here value full elements like US,  if i want to know why element are making that a country go high, i dont think that i can mix ihm with lower avlue country, because he will push others with wrong informations and other elements f the category will weak his importance

but i will try both for every models

.define bin_balanced same number in each category

In [ ]:
codiv_country_analyze['category_balanced'], bin_edges_balanced=pd.qcut(codiv_country_analyze['Total Infected Log10'], q=6,labels=False, retbins=True)
print(bin_edges_balanced)

In [ ]:
#explanation of these values
pd.qcut(codiv_country_analyze['Total Infected Log10'], q=6)

In [ ]:
#balanced bins deliver category for 0-5, euqidistant from 1-6, so i will add 1 to the category numbers
codiv_country_analyze['category_balanced'] = codiv_country_analyze['category_balanced'] +1

In [ ]:
ylevel0_balanced=pd.DataFrame(zip(bin_edges_balanced),columns=['Threshold'])['Threshold'][0]
ylevel1_balanced=pd.DataFrame(zip(bin_edges_balanced),columns=['Threshold'])['Threshold'][1]
ylevel2_balanced=pd.DataFrame(zip(bin_edges_balanced),columns=['Threshold'])['Threshold'][2]
ylevel3_balanced=pd.DataFrame(zip(bin_edges_balanced),columns=['Threshold'])['Threshold'][3]
ylevel4_balanced=pd.DataFrame(zip(bin_edges_balanced),columns=['Threshold'])['Threshold'][4]
ylevel5_balanced=pd.DataFrame(zip(bin_edges_balanced),columns=['Threshold'])['Threshold'][5]
ylevel6_balanced=pd.DataFrame(zip(bin_edges_balanced),columns=['Threshold'])['Threshold'][6]
Level1_balanced='%.3f<Infected<%.3f'%(ylevel0_balanced,ylevel1_balanced,)
Level2_balanced='%.3f<Infected<%.3f'%(ylevel1_balanced,ylevel2_balanced,)
Level3_balanced='%.3f<Infected<%.3f'%(ylevel2_balanced,ylevel3_balanced,)
Level4_balanced='%.3f<Infected<%.3f'%(ylevel3_balanced,ylevel4_balanced,)
Level5_balanced='%.3f<Infected<%.3f'%(ylevel4_balanced,ylevel5_balanced,)
Level6_balanced='%.3f<Infected<%.3f'%(ylevel5_balanced,ylevel6_balanced,)
print("Level1_balanced= ",Level1_balanced)
print("Level2_balanced= ",Level2_balanced)
print("Level3_balanced= ",Level3_balanced)
print("Level4_balanced= ",Level4_balanced)
print("Level5_balanced= ",Level5_balanced)
print("Level6_balanced= ",Level6_balanced)

In [ ]:
list(codiv_country_analyze['category_balanced'].value_counts())

define bin_equidistants category

In [ ]:
ymax=codiv_country_analyze['Total Infected Log10'].values.max()
ymax=ymax+ymax*0.001
print("ymax = ",ymax)
ymin=(codiv_country_analyze['Total Infected Log10'].values.min())
ymin=ymin-ymin*0.001
print("ymin = ",ymin)
yinterval=(ymax-ymin)/6
# 
ylevel0_equidistant = ymin
ylevel1_equidistant = ymin+yinterval*1
ylevel2_equidistant = ymin+yinterval*2
ylevel3_equidistant = ymin+yinterval*3
ylevel4_equidistant = ymin+yinterval*4
ylevel5_equidistant = ymin+yinterval*5
ylevel6_equidistant = ymin+yinterval*6

#
Level1_equidistant='%.3f<Infected<%.3f'%(ylevel0_equidistant,ylevel1_equidistant,)
Level2_equidistant='%.3f<Infected<%.3f'%(ylevel1_equidistant,ylevel2_equidistant,)
Level3_equidistant='%.3f<Infected<%.3f'%(ylevel2_equidistant,ylevel3_equidistant,)
Level4_equidistant='%.3f<Infected<%.3f'%(ylevel3_equidistant,ylevel4_equidistant,)
Level5_equidistant='%.3f<Infected<%.3f'%(ylevel4_equidistant,ylevel5_equidistant,)
Level6_equidistant='%.3f<Infected<%.3f'%(ylevel5_equidistant,ylevel6_equidistant,)
codiv_country_analyze['category_equidistant']=np.digitize(codiv_country_analyze['Total Infected Log10'].values,
                          bins=[ylevel0_equidistant,ylevel1_equidistant,ylevel2_equidistant,ylevel3_equidistant,ylevel4_equidistant,ylevel5_equidistant,ylevel6_equidistant])
print("Level1_equidistant= ",Level1_equidistant)
print("Level2_equidistant= ",Level2_equidistant)
print("Level3_equidistant= ",Level3_equidistant)
print("Level4_equidistant= ",Level4_equidistant)
print("Level5_equidistant= ",Level5_equidistant)
print("Level6_equidistant= ",Level6_equidistant)

Population 2020 was used to normalize some data,  Density will  also not be relevant,  but we have Urban Pop

In [ ]:
y_equidistant = codiv_country_analyze['category_equidistant']
y_balanced = codiv_country_analyze['category_balanced']
X = codiv_country_analyze.drop(['Total Infected Log10','category_balanced','category_equidistant'], axis=1) 
sns.countplot(y_balanced)

In [ ]:
sns.countplot(y_equidistant)

Input matrix
---

In [ ]:
X.sample(10)

splitting in training and test, these will be used for the confusion matrix

In [ ]:
from sklearn.model_selection import train_test_split
X_tr_equid, X_te_equid, y_tr_equid, y_te_equid = train_test_split(X, y_equidistant, test_size=0.15, random_state=1)
print('Train set equidistant:', X_tr_equid.shape, y_tr_equid.shape)
print('Testn set equidistant:', X_te_equid.shape, y_te_equid.shape)
X_tr_balan, X_te_balan, y_tr_balan, y_te_balan = train_test_split(X, y_balanced, test_size=0.15, random_state=1)
print('Train set balanced:', X_tr_balan.shape, y_tr_balan.shape)
print('Testn set balanced:', X_te_balan.shape, y_te_balan.shape)

In [ ]:
sns.countplot(y_te_equid)

In [ ]:
sns.countplot(y_te_balan)

---
Baseline classifier most frequent - equidistant bins -  Total Infected log10 feature
---

I will use the "most-frequent" baseline.

For the evaluation, I will not split into train/test sets as the number of data points is very low. Instead, I will report the cross-validated score.

To tune  the optimal depth of the tree, I will use cross validation with the default `cv=5` value from Scikit-learn: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

In [ ]:
from sklearn.model_selection import KFold
cv_strategy = KFold(n_splits=5, shuffle=False, random_state=None)

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y_equidistant)
dummy_clf.predict(X)
print("scores baseline mostfrequent classifier equidistant = ", dummy_clf.score(X, y_equidistant))

In [ ]:
df_resultat.at[df_resultat.index==0,"method"]="baseline_equidistant"
Baseline_training_scores = cross_validate(dummy_clf, X, y_equidistant, cv=cv_strategy)
df_resultat.at[df_resultat.index==0,"Cross-validation"]=np.mean(Baseline_training_scores['test_score'])
print("crossvalidation baseline mostfrequent classifier equidistant = ", np.mean(Baseline_training_scores['test_score']) )

with split for the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
dummy_clf= DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_tr_equid, y_tr_equid)
y_pred_equid=dummy_clf.predict(X_te_equid)
matrix = confusion_matrix(y_true=y_te_equid, y_pred=y_pred_equid)
print(matrix)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(dummy_clf, X_te_equid, y_te_equid, cmap=plt.cm.Blues,display_labels=[1,2,3,4,5,6])

the highest class in test datas are in class 4, 6/15 he recognized good 0.40

Baseline  most frequent - balanced bins - Total Infected log10 feature
---

In [ ]:
df_resultat.at[df_resultat.index==1,"method"]="baseline balanced bins"
Baseline_training_scores = cross_validate(dummy_clf, X, y_balanced, cv=cv_strategy)
df_resultat.at[df_resultat.index==1,"Cross-validation"]=np.mean(Baseline_training_scores['test_score'])
print("crossvalidation baseline mostfrequent classifier balanced = ", np.mean(Baseline_training_scores['test_score']) )

The confusion_matrix with split

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_tr_balan, y_tr_balan)
y_pred_balan=dummy_clf.predict(X_te_balan)

In [ ]:
matrix = confusion_matrix(y_true=y_te_balan, y_pred=y_pred_balan)
print(matrix)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(dummy_clf, X_te_balan, y_te_balan, cmap=plt.cm.Greens,display_labels=[1,2,3,4,5,6])

---
Decision Tree - equidistant bins - Total Infected log10 feature
---

In [ ]:
from sklearn.tree import DecisionTreeClassifier

Tuning the decision tree on the complete data X, y with cross-validation.

In [ ]:
depths=range(1,15)
scoring='accuracy'
cv_val_scores_list = []
cv_val_scores_std = []
cv_val_scores_mean = []
accuracy_scores = []
for depth in depths:
    DecisionTree = DecisionTreeClassifier(criterion='gini',  random_state=0, max_depth=depth)
    DecisionTree.fit(X, y_equidistant)
    cv_val_scores = cross_val_score(DecisionTree, X, y_equidistant, cv=cv_strategy, scoring=scoring)
    cv_val_scores_list.append(cv_val_scores)
    cv_val_scores_mean.append(cv_val_scores.mean())
    cv_val_scores_std.append(cv_val_scores.std())
    accuracy_scores.append(DecisionTree.score(X, y_equidistant))
    Text="depth %d, cv_val_scores_mean %f  score %f"%(depth,cv_val_scores.mean(),DecisionTree.score(X, y_equidistant))
    print(Text)
cv_val_scores_mean = np.array(cv_val_scores_mean)
cv_val_scores_std = np.array(cv_val_scores_std)
accuracy_scores = np.array(accuracy_scores)

In [ ]:
print("The maximum is by depth = ",np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1)
OptDepth=np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,5))
ax.plot(depths, cv_val_scores_mean, '-o', label='mean cross-validation accuracy', alpha=0.9)
ax.fill_between(depths, cv_val_scores_mean-2*cv_val_scores_std, cv_val_scores_mean+2*cv_val_scores_std, alpha=0.2)
ylim = plt.ylim()
ax.plot(depths, accuracy_scores, '-*', label='training_scores', alpha=0.9)
title="decision tree accurency"
ax.set_title(title, fontsize=16)
ax.set_xlabel('Tree depth', fontsize=14)
ax.set_ylabel('accuracy', fontsize=14)
ax.set_xticks(depths)
ax.legend()

computing the tree again with the optimal depth

In [ ]:
DecisionTree = DecisionTreeClassifier(
    criterion='gini',  random_state=0, max_depth=OptDepth)
# Fit decision tree
DecisionTree.fit(X, y_equidistant)
Tree_scores = cross_validate(DecisionTree, X, y_equidistant, cv=cv_strategy)
df_resultat.at[df_resultat.index==2,"method"]="decision-tree equidistant"
df_resultat.at[df_resultat.index==2,"Cross-validation"]=np.mean(Tree_scores['test_score'])
                                                              
print("score DecistionTree equidistant =",DecisionTree.score(X, y_equidistant))
print("cross validation DecisionTree equidistant =",np.mean(Tree_scores['test_score']))

confusion matrix with the optimal depth

In [ ]:
from sklearn.metrics import confusion_matrix
DecisionTree_confusion = DecisionTreeClassifier(criterion='gini',  random_state=0, max_depth=OptDepth)
DecisionTree_confusion.fit(X_tr_equid, y_tr_equid)
confusion_matrix(y_te_equid, DecisionTree_confusion.predict(X_te_equid))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(DecisionTree_confusion, X_te_equid, y_te_equid,cmap=plt.cm.Blues,display_labels=[1,2,3,4,5,6])

Display the tree

In [ ]:
from sklearn.tree import export_graphviz
import os
os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/'
# Export decision tree
dot_data = export_graphviz(
    DecisionTree, out_file=None,
    feature_names=X.columns, class_names=[Level1_equidistant,Level2_equidistant, Level3_equidistant,Level4_equidistant,Level5_equidistant,Level6_equidistant],
    filled=True, rounded=True, proportion=True   
)
import graphviz

# Display decision tree
graphviz.Source(dot_data)

---
Decision Tree - balanced bins - Total Infected log10 feature
---

In [ ]:
depths=range(1,15)
scoring='accuracy'
cv_val_scores_list = []
cv_val_scores_std = []
cv_val_scores_mean = []
accuracy_scores = []
for depth in depths:
    DecisionTree = DecisionTreeClassifier(criterion='gini',  random_state=0, max_depth=depth)
    DecisionTree.fit(X, y_balanced)
    cv_val_scores = cross_val_score(DecisionTree, X, y_balanced, cv=cv_strategy, scoring=scoring)
    cv_val_scores_list.append(cv_val_scores)
    cv_val_scores_mean.append(cv_val_scores.mean())
    cv_val_scores_std.append(cv_val_scores.std())
    accuracy_scores.append(DecisionTree.score(X, y_balanced))
    Text="depth %d, cv_val_scores_mean %f  score %f"%(depth,cv_val_scores.mean(),DecisionTree.score(X, y_balanced))
    print(Text)
cv_val_scores_mean = np.array(cv_val_scores_mean)
cv_val_scores_std = np.array(cv_val_scores_std)
accuracy_scores = np.array(accuracy_scores)

In [ ]:
print("The maximum is by depth_balanced = ",np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1)
OptDepth=np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,5))
ax.plot(depths, cv_val_scores_mean, '-o', label='mean cross-validation accuracy', alpha=0.9)
ax.fill_between(depths, cv_val_scores_mean-2*cv_val_scores_std, cv_val_scores_mean+2*cv_val_scores_std, alpha=0.2)
ylim = plt.ylim()
ax.plot(depths, accuracy_scores, '-*', label='training_scores', alpha=0.9)
title="decision tree accurency - _balanced bins"
ax.set_title(title, fontsize=16)
ax.set_xlabel('Tree depth', fontsize=14)
ax.set_ylabel('accuracy', fontsize=14)
ax.set_xticks(depths)
ax.legend()

In [ ]:
DecisionTree = DecisionTreeClassifier(
    criterion='gini',  random_state=0, max_depth=OptDepth)
# Fit decision tree
DecisionTree.fit(X, y_balanced)
# Get score
Tree_scores = cross_validate(DecisionTree, X, y_balanced, cv=cv_strategy)
df_resultat.at[df_resultat.index==3,"method"]="decision-tree _balanced bins"
df_resultat.at[df_resultat.index==3,"Cross-validation"]=np.mean(Tree_scores['test_score'])
print("score DecisionTree _balanced bins =",DecisionTree.score(X, y_balanced))
print("cross validation DecisionTree balanced bins=",np.mean(Tree_scores['test_score']))

Depending on the seed, the results can vary, but the optimal models always have a large depth (usually between 7-10).

The decision tree performs better than the baseline.

In [ ]:
from sklearn.metrics import confusion_matrix
DecisionTree_confusion = DecisionTreeClassifier(criterion='gini',  random_state=0, max_depth=OptDepth)
DecisionTree_confusion.fit(X_tr_balan, y_tr_balan)
confusion_matrix(y_te_balan, DecisionTree_confusion.predict(X_te_balan))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(DecisionTree_confusion, X_te_balan, y_te_balan,cmap=plt.cm.Greens,display_labels=[1,2,3,4,5,6])

In [ ]:
from sklearn.tree import export_graphviz
import os
os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/'
# Export decision tree
dot_data = export_graphviz(
    DecisionTree, out_file=None,
    feature_names=X.columns, class_names=[Level1_balanced,Level2_balanced, Level3_balanced,Level4_balanced,Level5_balanced,Level6_balanced],
    filled=True, rounded=True, proportion=True   
)
import graphviz

# Display decision tree
graphviz.Source(dot_data)

---
Random forests - equidistant bins - Total Infected log10 feature
---

I will use the same methodology as above. I use 250 for the number of `n_estimators` as this seems to be a good default value in practice from my research on online forums.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
Estimators=range(100,2000,100)
scoring='accuracy'
cv_val_scores_list = []
cv_val_scores_std = []
cv_val_scores_mean = []
accuracy_scores = []
for Estimator in Estimators:
    forest = RandomForestClassifier(n_estimators=Estimator,random_state=0)
    forest.fit(X, y_equidistant)
    cv_val_scores = cross_val_score(forest, X, y_equidistant, cv=cv_strategy, scoring=scoring)
    cv_val_scores_list.append(cv_val_scores)
    cv_val_scores_mean.append(cv_val_scores.mean())
    cv_val_scores_std.append(cv_val_scores.std())
    accuracy_scores.append(forest.fit(X, y_equidistant).score(X, y_equidistant))
    Text="depth %d, cv_val_scores_mean %f  score %f"%(Estimator,cv_val_scores.mean(),forest.score(X, y_equidistant))
    print(Text)
cv_val_scores_mean = np.array(cv_val_scores_mean)
cv_val_scores_std = np.array(cv_val_scores_std)
accuracy_scores = np.array(accuracy_scores)

In [ ]:
print("The maximum is by Estimator_balanced = ",(np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1)*100)
OptEstimator=(np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1)*100

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,5))
ax.plot(Estimators, cv_val_scores_mean, '-o', label='mean cross-validation accuracy', alpha=0.9)
ax.fill_between(Estimators, cv_val_scores_mean-2*cv_val_scores_std, cv_val_scores_mean+2*cv_val_scores_std, alpha=0.2)
ylim = plt.ylim()
ax.plot(Estimators, accuracy_scores, '-*', label='training_scores', alpha=0.9)
title="Forest accurency - _balanced bins"
ax.set_title(title, fontsize=16)
ax.set_xlabel('Forest Estimator', fontsize=14)
ax.set_ylabel('accuracy', fontsize=14)
ax.set_xticks(Estimators)
ax.legend()

compute for the optimal estimator

In [ ]:
# Build a forest and compute the feature importances
forest = RandomForestClassifier(n_estimators=OptEstimator,random_state=0)
forest.fit(X, y_equidistant)

In [ ]:
# Mean test score of a 250x decision tree 
Forest_scores = cross_validate(forest, X, y_equidistant, cv=cv_strategy)
print('Forest by equidistant interval score {:.3f}'.format(forest.score(X, y_equidistant)))
df_resultat.at[df_resultat.index==4,"method"]="Forest equidistant"
df_resultat.at[df_resultat.index==4,"Cross-validation"]=np.mean(Forest_scores['test_score'])
print('Forest by equidistant interval cross validation {:.3f}'.format(np.mean(Forest_scores['test_score'])))

confusion matrix  with split data

In [ ]:
forest_confusion = RandomForestClassifier(n_estimators=OptEstimator,random_state=0)
forest_confusion.fit(X_tr_equid, y_tr_equid)
confusion_matrix(y_te_equid, forest_confusion.predict(X_te_equid))

In [ ]:
plot_confusion_matrix(forest_confusion, X_te_equid,y_te_equid,cmap=plt.cm.Blues,display_labels=[1,2,3,4,5,6])

features importances

In [ ]:
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],axis=0)
indices = np.argsort(importances)[::-1]

In [ ]:
# Print the feature ranking
print("Feature ranking by equidistand interval:")

for f in range(X.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, X.columns[indices[f]], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure(figsize=(20,10))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation = 65)
plt.xlim([-1, X.shape[1]])
plt.show()

Those results make sense as the "Urban Population" is directly affecting how much the epidemic is spreading. Also, if the population is older, then it will also affect the number of people that are detected.

I was maybe expecting the "Smoking" to be higher, but it's difficult to know its effect on the disease.

Regarding the quarantine measures, the quarantine/schools/restrictions are at the end of the results, but I think that this is because it highly depends on how this is applied. For instance, in China or Germany, people going to the supermarket have to wear masks which is not the case for instance in Switzerland or other countries ex. in Europe. Distance measures are also not the same.

---
Random forest - Balanced bins - Total Infected log10 feature
---

In [ ]:
Estimators=range(100,2000,100)
scoring='accuracy'
cv_val_scores_list = []
cv_val_scores_std = []
cv_val_scores_mean = []
accuracy_scores = []
for Estimator in Estimators:
    forest = RandomForestClassifier(n_estimators=Estimator,random_state=0)
    forest.fit(X, y_balanced)
    cv_val_scores = cross_val_score(forest, X, y_balanced, cv=cv_strategy, scoring=scoring)
    cv_val_scores_list.append(cv_val_scores)
    cv_val_scores_mean.append(cv_val_scores.mean())
    cv_val_scores_std.append(cv_val_scores.std())
    accuracy_scores.append(forest.fit(X, y_balanced).score(X, y_balanced))
    Text="depth %d, cv_val_scores_mean %f  score %f"%(Estimator,cv_val_scores.mean(),forest.score(X, y_balanced))
    print(Text)
cv_val_scores_mean = np.array(cv_val_scores_mean)
cv_val_scores_std = np.array(cv_val_scores_std)
accuracy_scores = np.array(accuracy_scores)

In [ ]:
print("The maximum is by Estimator_balanced = ",(np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1)*100)
OptEstimator=(np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1)*100

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,5))
ax.plot(Estimators, cv_val_scores_mean, '-o', label='mean cross-validation accuracy', alpha=0.9)
ax.fill_between(Estimators, cv_val_scores_mean-2*cv_val_scores_std, cv_val_scores_mean+2*cv_val_scores_std, alpha=0.2)
ylim = plt.ylim()
ax.plot(Estimators, accuracy_scores, '-*', label='training_scores', alpha=0.9)
title="Forest accurency - _balanced bins"
ax.set_title(title, fontsize=16)
ax.set_xlabel('Forest Estimator', fontsize=14)
ax.set_ylabel('accuracy', fontsize=14)
ax.set_xticks(Estimators)
ax.legend()

In [ ]:
from sklearn.model_selection import cross_validate
Forest_scores = cross_validate(forest, X, y_balanced, cv=cv_strategy)
df_resultat.at[df_resultat.index==5,"method"]="Forest balanced bins"
df_resultat.at[df_resultat.index==5,"Cross-validation"]=np.mean(Forest_scores['test_score'])
print('Forest by balanced bins of element score {:.3f}'.format(forest.score(X, y_balanced)))
print('Forest by balanced bins of element - mean test {:.3f}'.format(np.mean(Forest_scores['test_score'])))

confusion matrix

In [ ]:
forest_confusion = RandomForestClassifier(n_estimators=OptEstimator,random_state=0)
forest_confusion.fit(X_tr_balan, y_tr_balan)
confusion_matrix(y_te_balan, forest_confusion.predict(X_te_balan))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(forest_confusion, X_te_balan, y_te_balan,cmap=plt.cm.Blues,display_labels=[1,2,3,4,5,6])

Compute feature importance

In [ ]:
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

In [ ]:
# Print the feature ranking
print("Feature ranking by balanced bins:")

for f in range(X.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, X.columns[indices[f]], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure(figsize=(20,10))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation = 65)
plt.xlim([-1, X.shape[1]])
plt.show()

I think that this highlight an issue with the "balanced" approach: we are mixing together countries that have a very different number of infected people, in particular with the top bins ex. category 5 and 6.

---
Feature reduction to 2 and kneighbour classifier - equidistant bins - Total Infected log10 feature
---
feature reduction

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
# Apply PCA
pca.fit(X, y=None); # Unsupervised learning, no y variable
feature_2= pca.transform(X)
feature_2_df=pd.DataFrame(feature_2)
len_cluster, col=feature_2_df.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_fea_2_tr_equid, X_fea_2_te_equid, y_fea_2_tr_equid, y_fea_2_te_equid = train_test_split(feature_2, y_equidistant, test_size=0.15, random_state=1)
print('Train set equidistant:', X_fea_2_tr_equid.shape, y_fea_2_tr_equid.shape)
print('Testn set equidistant:', X_fea_2_te_equid.shape, y_fea_2_te_equid.shape)
X_fea_2_tr_balan, X_fea_2_te_balan, y_fea_2_tr_balan, y_fea_2_te_balan = train_test_split(feature_2, y_balanced, test_size=0.15, random_state=1)
print('Train set balanced:', X_fea_2_tr_balan.shape, y_fea_2_tr_balan.shape)
print('Testn set balanced:', X_fea_2_te_balan.shape, y_fea_2_te_balan.shape)

---
K neightbour classifier - equidistant bins - Total Infected log10 feature
---

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Create k-NN classifier
pipe = Pipeline([
    #('scaler', StandardScaler()), # With standardization
    ('scaler', None), # Better performance without standardization!
    ('knn', KNeighborsClassifier(
        n_jobs=-1 # As many parallel jobs as possible
    ))
])


i will use the weight distance   because i cant explain the result default=’uniform’

‘uniform’ : uniform weights. All points in each neighborhood are weighted equally.

‘distance’ : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away

In [ ]:
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
from sklearn import metrics
from sklearn.model_selection import cross_val_score
neighbors = list(range(1,35))
CV_scores = []


for n_neighborss in neighbors:
    weights='distance'
    feature1=pd.DataFrame(feature_2).loc[:][0]
    feature2=pd.DataFrame(feature_2).loc[:][1]
    h = .02  # step size in the mesh
    # we create an instance of Neighbours Classifier and fit the data.
    # Create a k-NN pipeline
    knn_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('knn', KNeighborsClassifier(n_neighbors=n_neighborss, weights=weights)) ])
    # Fit estimator
    knn_pipe.fit(feature_2, y_equidistant)
    scores = cross_val_score(knn_pipe,feature_2,y_equidistant,cv=cv_strategy )
    CV_scores.append(scores.mean())
Missclassification_Error = [1-x for x in CV_scores]
print("the optimal k is ",format(Missclassification_Error.index(min(Missclassification_Error))))
k_Optimal=Missclassification_Error.index(min(Missclassification_Error))

In [ ]:
from matplotlib.legend_handler import HandlerLine2D
plt.plot(Missclassification_Error)
plt.xlabel("number of neighbors K")
plt.ylabel("Missclassification Error")
plt.show()

In [ ]:
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
from sklearn.metrics import accuracy_score

# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF', '#EFFF00', '#B4FCFF', '#FFB4F5'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF', '#FF00F7', '#00FFE6', '#FFA200'])

feature1=pd.DataFrame(feature_2).loc[:][0]
feature2=pd.DataFrame(feature_2).loc[:][1]

weights='distance'
h = .02  # step size in the mesh
# we create an instance of Neighbours Classifier and fit the data.
# Create a k-NN pipeline
knn_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=k_Optimal))
    #, weights=weights))
])
# Fit estimator
knn_pipe.fit(feature_2, y_equidistant)
# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
feature1_min, feature1_max = feature1.min() - 1, feature1.max() + 1
feature2_min, feature2_max = feature2.min() - 1, feature2.max() + 1
xx, yy = np.meshgrid(np.arange(feature1_min, feature1_max, h), np.arange(feature2_min, feature2_max, h))
Z = knn_pipe.predict(np.c_[xx.ravel(), yy.ravel()])
Zo = Z.reshape(xx.shape)

In [ ]:
# Put the result into a color plot
plt.figure(figsize=(18, 10))
plt.pcolormesh(xx, yy, Zo, cmap=cmap_light)
#Label1=pd.DataFrame(list(class_index.items()))
Label1=pd.DataFrame([1,2,3,4,5])
# Plot also the training points
plt.scatter(feature1, feature2, c=y_equidistant, cmap=cmap_bold)
#color bar with label
cbar = plt.colorbar() 
cbar.ax.set_yticklabels([Level1_equidistant,Level2_equidistant,Level3_equidistant,Level4_equidistant,Level5_equidistant,Level6_equidistant])
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xlabel("feature 1")
plt.ylabel("feature 2")
plt.title("6-Class classification,   equidistant bins -  total infected log10 (k = %i, weights = '%s')" % (k_Optimal, weights))
plt.show()

score and cross validation

In [ ]:
from sklearn.model_selection import cross_validate
knn_scores = cross_validate(knn_pipe, feature_2, y_equidistant, cv=cv_strategy)
df_resultat.at[df_resultat.index==6,"method"]="knn equidistant bins"
df_resultat.at[df_resultat.index==6,"Cross-validation"]=np.mean(knn_scores['test_score'])
print('knn by equidistant bins of element score {:.3f}'.format(knn_pipe.score(feature_2, y_equidistant)))
print('knn by equidistant bins of element - mean test {:.3f}'.format(np.mean(knn_scores['test_score'])))

For the confusion matrix i use the split data

In [ ]:
knn_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=k_Optimal, weights=weights))
])
# Fit estimator
knn_confusion=knn_pipe.fit(X_fea_2_tr_equid, y_fea_2_tr_equid)
knn_predict = knn_pipe.predict(X_fea_2_te_equid)
confusion_matrix(y_fea_2_te_equid, knn_predict)

In [ ]:
plot_confusion_matrix(knn_confusion, X_fea_2_te_equid, y_fea_2_te_equid,cmap=plt.cm.Blues,display_labels=[1,2,3,4,5,6])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=y_fea_2_te_equid, y_pred=knn_predict))

---
K neightbour classifier - balanced bins - Total Infected log10 feature
---
optimize the n_neighbors, over 35 i am getting cpu problem
higher is the n_neighbors more is he isolating every point, it means high n_neighbors would only means there is no big area available with same label elements

In [ ]:
neighbors = list(range(1,35))
CV_scores = []

for n_neighborss in neighbors:
    weights='distance'
    feature1=pd.DataFrame(feature_2).loc[:][0]
    feature2=pd.DataFrame(feature_2).loc[:][1]
    h = .02  # step size in the mesh
    # we create an instance of Neighbours Classifier and fit the data.
    # Create a k-NN pipeline
    knn_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('knn', KNeighborsClassifier(n_neighbors=n_neighborss, weights=weights))
    ])
    # Fit estimator
    knn_pipe.fit(feature_2, y_balanced)
    #
    scores = cross_val_score(knn_pipe,feature_2,y_balanced,cv=cv_strategy )
    CV_scores.append(scores.mean())
    
Missclassification_Error = [1-x for x in CV_scores]
print("the optimal k is ",format(Missclassification_Error.index(min(Missclassification_Error))))
k_Optimal=Missclassification_Error.index(min(Missclassification_Error))
    
from matplotlib.legend_handler import HandlerLine2D
plt.plot( Missclassification_Error)
plt.xlabel("number of neighbors K")
plt.ylabel("Missclassification Error")
plt.show()

In [ ]:
# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF', '#EFFF00', '#B4FCFF', '#FFB4F5'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF', '#FF00F7', '#00FFE6', '#FFA200'])
#
feature1=pd.DataFrame(feature_2).loc[:][0]
feature2=pd.DataFrame(feature_2).loc[:][1]
#
weights='distance'
h = .02  # step size in the mesh
# Create a k-NN pipeline
knn_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=k_Optimal, weights=weights))
])
# Fit estimator
knn_pipe.fit(feature_2, y_balanced)
# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
feature1_min, feature1_max = feature1.min() - 1, feature1.max() + 1
feature2_min, feature2_max = feature2.min() - 1, feature2.max() + 1
xx, yy = np.meshgrid(np.arange(feature1_min, feature1_max, h), np.arange(feature2_min, feature2_max, h))
Z = knn_pipe.predict(np.c_[xx.ravel(), yy.ravel()])
Zo = Z.reshape(xx.shape)

In [ ]:
# Put the result into a color plot
plt.figure(figsize=(18, 10))
plt.pcolormesh(xx, yy, Zo, cmap=cmap_light)
#Label1=pd.DataFrame(list(class_index.items()))
Label1=pd.DataFrame([1,2,3,4,5])
# Plot also the training points
plt.scatter(feature1, feature2, c=y_balanced, cmap=cmap_bold)
cbar = plt.colorbar() 
cbar.ax.set_yticklabels([Level1_balanced,Level2_balanced,Level3_balanced,Level4_balanced,Level5_balanced,Level6_balanced])
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xlabel("feature 1")
plt.ylabel("feature 2")
plt.title("6-Class classification,   balanced bins -  total infected log10 (k = %i, weights = '%s')" % (k_Optimal, weights))

plt.show()

In [ ]:
from sklearn.model_selection import cross_validate
knn_scores = cross_validate(knn_pipe, feature_2, y_balanced, cv=cv_strategy)
df_resultat.at[df_resultat.index==7,"method"]="knn balanced bins"
df_resultat.at[df_resultat.index==7,"Cross-validation"]=np.mean(knn_scores['test_score'])
print('knn by equidistant bins of element score {:.3f}'.format(knn_pipe.score(feature_2, y_balanced)))
print('knn by equidistant bins of element - mean test_score {:.3f}'.format(np.mean(knn_scores['test_score'])))

In [ ]:
knn_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=k_Optimal, weights=weights))
])
# Fit estimator
knn_confusion=knn_pipe.fit(X_fea_2_tr_balan, y_fea_2_tr_balan)
knn_predict = knn_pipe.predict(X_fea_2_te_balan)

In [ ]:
confusion_matrix(y_fea_2_te_balan, knn_predict)

In [ ]:
plot_confusion_matrix(knn_confusion, X_fea_2_te_balan, y_fea_2_te_balan,cmap=plt.cm.Greens,display_labels=[1,2,3,4,5,6])

---
summary - Total Infected log10 feature
---

In [ ]:
df_resultat

In [ ]:
sns.barplot(x=df_resultat["Cross-validation"],y=df_resultat['method'])
plt.show()

Equidistant bins deliver better resultat
---
---
lets analyze  the deaths ratio feature 
---
codiv_csse["Deaths Ratio"]=codiv_csse["Deaths"]*1000/codiv_csse["Infected"]

In [ ]:
# Working on a copy
codiv_country_analyze = codiv_country.copy()

# Creating categorical variables
codiv_country_analyze['Quarantine_cat'] = codiv_country_analyze['Quarantine'].notnull().astype(int)
codiv_country_analyze['Restrictions_cat'] = codiv_country_analyze['Restrictions'].notnull().astype(int)
codiv_country_analyze['Schools_cat'] = codiv_country_analyze['Schools'].notnull().astype(int)

# 
codiv_country_analyze=codiv_country_analyze.drop([
    'Quarantine','Schools','Restrictions', # now categorical
    'Country', # not helpful
    'Total Deaths','Total Infected','Total Active','Total Recovered', 
    "Total Deaths Log10","Total Recovered Log10","Total Active Log10","Total Infected Log10"
], axis=1)

codiv_country_analyze.sample(10)

In [ ]:
columns = ['method','Cross-validation']
index=range(8)
df_resultat = pd.DataFrame(index=index, columns=columns)

Balanced bins

In [ ]:
codiv_country_analyze['Deaths Ratio'].describe()

In [ ]:
codiv_country_analyze['category_balanced'], bin_edges_balanced=pd.qcut(codiv_country_analyze['Deaths Ratio'], q=6,labels=False, retbins=True)
#balanced bins deliver category for 0-5, euqidistant from 1-6, so i will add 1 to the category numbers
codiv_country_analyze['category_balanced'] = codiv_country_analyze['category_balanced'] +1

In [ ]:
ylevel0_balanced=pd.DataFrame(zip(bin_edges_balanced),columns=['Threshold'])['Threshold'][0]
ylevel1_balanced=pd.DataFrame(zip(bin_edges_balanced),columns=['Threshold'])['Threshold'][1]
ylevel2_balanced=pd.DataFrame(zip(bin_edges_balanced),columns=['Threshold'])['Threshold'][2]
ylevel3_balanced=pd.DataFrame(zip(bin_edges_balanced),columns=['Threshold'])['Threshold'][3]
ylevel4_balanced=pd.DataFrame(zip(bin_edges_balanced),columns=['Threshold'])['Threshold'][4]
ylevel5_balanced=pd.DataFrame(zip(bin_edges_balanced),columns=['Threshold'])['Threshold'][5]
ylevel6_balanced=pd.DataFrame(zip(bin_edges_balanced),columns=['Threshold'])['Threshold'][6]
Level1_balanced='%.3f<Deaths Ratio<%.3f'%(ylevel0_balanced,ylevel1_balanced,)
Level2_balanced='%.3f<Deaths Ratio<%.3f'%(ylevel1_balanced,ylevel2_balanced,)
Level3_balanced='%.3f<Deaths Ratio<%.3f'%(ylevel2_balanced,ylevel3_balanced,)
Level4_balanced='%.3f<Deaths Ratio<%.3f'%(ylevel3_balanced,ylevel4_balanced,)
Level5_balanced='%.3f<Deaths Ratio<%.3f'%(ylevel4_balanced,ylevel5_balanced,)
Level6_balanced='%.3f<Deaths Ratio<%.3f'%(ylevel5_balanced,ylevel6_balanced,)
print("Level1_balanced= ",Level1_balanced)
print("Level2_balanced= ",Level2_balanced)
print("Level3_balanced= ",Level3_balanced)
print("Level4_balanced= ",Level4_balanced)
print("Level5_balanced= ",Level5_balanced)
print("Level6_balanced= ",Level6_balanced)

equidistant bins

In [ ]:
ymax=codiv_country_analyze['Deaths Ratio'].values.max()
print(ymax)
ymax=ymax+ymax*0.001
print("ymax = ",ymax)
ymin=(codiv_country_analyze['Deaths Ratio'].values.min())
ymin=ymin-ymin*0.001
print("ymin = ",ymin)
yinterval=(ymax-ymin)/6
print(ymin,ymax)
# 
ylevel0_equidistant = ymin
ylevel1_equidistant = ymin+yinterval*1
ylevel2_equidistant = ymin+yinterval*2
ylevel3_equidistant = ymin+yinterval*3
ylevel4_equidistant = ymin+yinterval*4
ylevel5_equidistant = ymin+yinterval*5
ylevel6_equidistant = ymin+yinterval*6

#
Level1_equidistant='%.3f<Deaths Ratio<%.3f'%(ylevel0_equidistant,ylevel1_equidistant,)
Level2_equidistant='%.3f<Deaths Ratio<%.3f'%(ylevel1_equidistant,ylevel2_equidistant,)
Level3_equidistant='%.3f<Deaths Ratio<%.3f'%(ylevel2_equidistant,ylevel3_equidistant,)
Level4_equidistant='%.3f<Deaths Ratio<%.3f'%(ylevel3_equidistant,ylevel4_equidistant,)
Level5_equidistant='%.3f<Deaths Ratio<%.3f'%(ylevel4_equidistant,ylevel5_equidistant,)
Level6_equidistant='%.3f<Deaths Ratio<%.3f'%(ylevel5_equidistant,ylevel6_equidistant,)
codiv_country_analyze['category_equidistant']=np.digitize(codiv_country_analyze['Deaths Ratio'].values,
                          bins=[ylevel0_equidistant,ylevel1_equidistant,ylevel2_equidistant,ylevel3_equidistant,ylevel4_equidistant,ylevel5_equidistant,ylevel6_equidistant])
print("Level1_equidistant= ",Level1_equidistant)
print("Level2_equidistant= ",Level2_equidistant)
print("Level3_equidistant= ",Level3_equidistant)
print("Level4_equidistant= ",Level4_equidistant)
print("Level5_equidistant= ",Level5_equidistant)
print("Level6_equidistant= ",Level6_equidistant)

In [ ]:
y_equidistant = codiv_country_analyze['category_equidistant']
y_balanced = codiv_country_analyze['category_balanced']
X = codiv_country_analyze.drop(['Deaths Ratio','category_balanced','category_equidistant'], axis=1) 
sns.countplot(y_balanced)

In [ ]:
sns.countplot(y_equidistant)

In [ ]:
from sklearn.model_selection import train_test_split
X_tr_equid, X_te_equid, y_tr_equid, y_te_equid = train_test_split(X, y_equidistant, test_size=0.15, random_state=1)
print('Train set equidistant:', X_tr_equid.shape, y_tr_equid.shape)
print('Testn set equidistant:', X_te_equid.shape, y_te_equid.shape)
X_tr_balan, X_te_balan, y_tr_balan, y_te_balan = train_test_split(X, y_balanced, test_size=0.15, random_state=1)
print('Train set balanced:', X_tr_balan.shape, y_tr_balan.shape)
print('Testn set balanced:', X_te_balan.shape, y_te_balan.shape)

In [ ]:
sns.countplot(y_te_equid)

In [ ]:
sns.countplot(y_te_balan)

---
Baseline classifier - most frequent - Deaths Ratio feature
---

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y_equidistant)
dummy_clf.predict(X)
print("scores baseline mostfrequent classifier equidistant = ", dummy_clf.score(X, y_equidistant))

In [ ]:
df_resultat.at[df_resultat.index==0,"method"]="baseline_equidistant"
Baseline_training_scores = cross_validate(dummy_clf, X, y_equidistant, cv=cv_strategy)
df_resultat.at[df_resultat.index==0,"Cross-validation"]=np.mean(Baseline_training_scores['test_score'])
print("crossvalidation baseline mostfrequent classifier equidistant = ", np.mean(Baseline_training_scores['test_score']) )

In [ ]:
from sklearn.metrics import confusion_matrix
dummy_clf= DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_tr_equid, y_tr_equid)
y_pred_equid=dummy_clf.predict(X_te_equid)
matrix = confusion_matrix(y_true=y_te_equid, y_pred=y_pred_equid)
print(matrix)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(dummy_clf, X_te_equid, y_te_equid, cmap=plt.cm.Blues,display_labels=[1,2,3,4,5,6])

Baseline  most frequent - balanced bins  - Deaths Ratio feature
---

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y_balanced)
dummy_clf.predict(X)
print("training_scores baseline mostfrequent classifier balanced = ", dummy_clf.score(X, y_balanced))

In [ ]:
df_resultat.at[df_resultat.index==1,"method"]="baseline balanced bins"
Baseline_training_scores = cross_validate(dummy_clf, X, y_balanced, cv=cv_strategy)
df_resultat.at[df_resultat.index==1,"Cross-validation"]=np.mean(Baseline_training_scores['test_score'])
print("crossvalidation baseline mostfrequent classifier balanced = ", np.mean(Baseline_training_scores['test_score']) )

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_tr_balan, y_tr_balan)
y_pred_balan=dummy_clf.predict(X_te_balan)

In [ ]:
matrix = confusion_matrix(y_true=y_te_balan, y_pred=y_pred_balan)
print(matrix)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(dummy_clf, X_te_balan, y_te_balan, cmap=plt.cm.Greens,display_labels=[1,2,3,4,5,6])

---
Decision Tree - equidistant bins  - Deaths Ratio feature
---

In [ ]:
depths=range(1,15)
scoring='accuracy'
cv_val_scores_list = []
cv_val_scores_std = []
cv_val_scores_mean = []
accuracy_scores = []
for depth in depths:
    DecisionTree = DecisionTreeClassifier(criterion='gini',  random_state=0, max_depth=depth)
    DecisionTree.fit(X, y_equidistant)
    cv_val_scores = cross_val_score(DecisionTree, X, y_equidistant, cv=cv_strategy, scoring=scoring)
    cv_val_scores_list.append(cv_val_scores)
    cv_val_scores_mean.append(cv_val_scores.mean())
    cv_val_scores_std.append(cv_val_scores.std())
    accuracy_scores.append(DecisionTree.score(X, y_equidistant))
    Text="depth %d, cv_val_scores_mean %f std %f score %f"%(depth,cv_val_scores.mean(),cv_val_scores.std(),DecisionTree.score(X, y_equidistant))
    print(Text)
cv_val_scores_mean = np.array(cv_val_scores_mean)
cv_val_scores_std = np.array(cv_val_scores_std)
accuracy_scores = np.array(accuracy_scores)

In [ ]:
print("The maximum is by depth = ",np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1)
OptDepth=np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,5))
ax.plot(depths, cv_val_scores_mean, '-o', label='mean cross-validation accuracy', alpha=0.9)
ax.fill_between(depths, cv_val_scores_mean-2*cv_val_scores_std, cv_val_scores_mean+2*cv_val_scores_std, alpha=0.2)
ylim = plt.ylim()
ax.plot(depths, accuracy_scores, '-*', label='training_scores', alpha=0.9)
title="decision tree accurency"
ax.set_title(title, fontsize=16)
ax.set_xlabel('Tree depth', fontsize=14)
ax.set_ylabel('accuracy', fontsize=14)
ax.set_xticks(depths)
ax.legend()

In [ ]:
DecisionTree = DecisionTreeClassifier(
    criterion='gini',  random_state=0, max_depth=OptDepth)
# Fit decision tree
DecisionTree.fit(X, y_equidistant)
Tree_scores = cross_validate(DecisionTree, X, y_equidistant, cv=cv_strategy)
df_resultat.at[df_resultat.index==2,"method"]="decision-tree equidistant"
df_resultat.at[df_resultat.index==2,"Cross-validation"]=np.mean(Tree_scores['test_score'])
                                                              
print("score DecistionTree equidistant =",DecisionTree.score(X, y_equidistant))
print("cross validation DecisionTree equidistant =",np.mean(Tree_scores['test_score']))

confusion matrix with the optimal depth

In [ ]:
from sklearn.metrics import confusion_matrix
DecisionTree_confusion = DecisionTreeClassifier(criterion='gini',  random_state=0, max_depth=OptDepth)
DecisionTree_confusion.fit(X_tr_equid, y_tr_equid)
confusion_matrix(y_te_equid, DecisionTree_confusion.predict(X_te_equid))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(DecisionTree_confusion, X_te_equid, y_te_equid,cmap=plt.cm.Blues,display_labels=[1,2,3,4,5,6])

In [ ]:
os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/'
# Export decision tree
dot_data = export_graphviz(
    DecisionTree, out_file=None,
    feature_names=X.columns, class_names=[Level1_equidistant,Level2_equidistant, Level3_equidistant,Level4_equidistant,Level5_equidistant,Level6_equidistant],
    filled=True, rounded=True, proportion=True   
)
import graphviz

# Display decision tree
graphviz.Source(dot_data)

Decision Tree - balanced bins  - Deaths Ratio feature
---

In [ ]:
depths=range(1,15)
scoring='accuracy'
cv_val_scores_list = []
cv_val_scores_std = []
cv_val_scores_mean = []
accuracy_scores = []
for depth in depths:
    DecisionTree = DecisionTreeClassifier(criterion='gini',  random_state=0, max_depth=depth)
    DecisionTree.fit(X, y_balanced)
    cv_val_scores = cross_val_score(DecisionTree, X, y_balanced, cv=cv_strategy, scoring=scoring)
    cv_val_scores_list.append(cv_val_scores)
    cv_val_scores_mean.append(cv_val_scores.mean())
    cv_val_scores_std.append(cv_val_scores.std())
    accuracy_scores.append(DecisionTree.score(X, y_balanced))
    Text="depth %d, cv_val_scores_mean %f  score %f"%(depth,cv_val_scores.mean(),DecisionTree.score(X, y_balanced))
    print(Text)
cv_val_scores_mean = np.array(cv_val_scores_mean)
cv_val_scores_std = np.array(cv_val_scores_std)
accuracy_scores = np.array(accuracy_scores)

In [ ]:
print("The maximum is by depth_balanced = ",np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1)
OptDepth=np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,5))
ax.plot(depths, cv_val_scores_mean, '-o', label='mean cross-validation accuracy', alpha=0.9)
ax.fill_between(depths, cv_val_scores_mean-2*cv_val_scores_std, cv_val_scores_mean+2*cv_val_scores_std, alpha=0.2)
ylim = plt.ylim()
ax.plot(depths, accuracy_scores, '-*', label='training_scores', alpha=0.9)
title="decision tree accurency - _balanced bins"
ax.set_title(title, fontsize=16)
ax.set_xlabel('Tree depth', fontsize=14)
ax.set_ylabel('accuracy', fontsize=14)
ax.set_xticks(depths)
ax.legend()

In [ ]:
DecisionTree = DecisionTreeClassifier(
    criterion='gini',  random_state=0, max_depth=OptDepth)
# Fit decision tree
DecisionTree.fit(X, y_balanced)
# Get score
Tree_scores = cross_validate(DecisionTree, X, y_balanced, cv=cv_strategy)
df_resultat.at[df_resultat.index==3,"method"]="decision-tree _balanced bins"
df_resultat.at[df_resultat.index==3,"Cross-validation"]=np.mean(Tree_scores['test_score'])
print("score DecisionTree _balanced bins =",DecisionTree.score(X, y_balanced))
print("cross validation DecisionTree balanced bins=",np.mean(Tree_scores['test_score']))

In [ ]:
from sklearn.metrics import confusion_matrix
DecisionTree_confusion = DecisionTreeClassifier(criterion='gini',  random_state=0, max_depth=OptDepth)
DecisionTree_confusion.fit(X_tr_balan, y_tr_balan)
confusion_matrix(y_te_balan, DecisionTree_confusion.predict(X_te_balan))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(DecisionTree_confusion, X_te_balan, y_te_balan,cmap=plt.cm.Greens,display_labels=[1,2,3,4,5,6])

In [ ]:
from sklearn.tree import export_graphviz
import os
os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/'
# Export decision tree
dot_data = export_graphviz(
    DecisionTree, out_file=None,
    feature_names=X.columns, class_names=[Level1_balanced,Level2_balanced, Level3_balanced,Level4_balanced,Level5_balanced,Level6_balanced],
    filled=True, rounded=True, proportion=True   
)
import graphviz

# Display decision tree
graphviz.Source(dot_data)

---
Random forests - equidistant bins  - Deaths Ratio feature
---

I will use the same methodology as above. I use 250 for the number of `n_estimators` as this seems to be a good default value in practice from my research on online forums.


In [ ]:
Estimators=range(100,2000,100)
scoring='accuracy'
cv_val_scores_list = []
cv_val_scores_std = []
cv_val_scores_mean = []
accuracy_scores = []
for Estimator in Estimators:
    forest = RandomForestClassifier(n_estimators=Estimator,random_state=0)
    forest.fit(X, y_equidistant)
    cv_val_scores = cross_val_score(forest, X, y_equidistant, cv=cv_strategy, scoring=scoring)
    cv_val_scores_list.append(cv_val_scores)
    cv_val_scores_mean.append(cv_val_scores.mean())
    cv_val_scores_std.append(cv_val_scores.std())
    accuracy_scores.append(forest.fit(X, y_equidistant).score(X, y_equidistant))
    Text="depth %d, cv_val_scores_mean %f  score %f"%(Estimator,cv_val_scores.mean(),forest.score(X, y_equidistant))
    print(Text)
cv_val_scores_mean = np.array(cv_val_scores_mean)
cv_val_scores_std = np.array(cv_val_scores_std)
accuracy_scores = np.array(accuracy_scores)

In [ ]:
print("The maximum is by Estimator_balanced = ",(np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1)*100)
OptEstimator=(np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1)*100

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,5))
ax.plot(Estimators, cv_val_scores_mean, '-o', label='mean cross-validation accuracy', alpha=0.9)
ax.fill_between(Estimators, cv_val_scores_mean-2*cv_val_scores_std, cv_val_scores_mean+2*cv_val_scores_std, alpha=0.2)
ylim = plt.ylim()
ax.plot(Estimators, accuracy_scores, '-*', label='training_scores', alpha=0.9)
title="Forest accurency - _balanced bins"
ax.set_title(title, fontsize=16)
ax.set_xlabel('Forest Estimator', fontsize=14)
ax.set_ylabel('accuracy', fontsize=14)
ax.set_xticks(Estimators)
ax.legend()

Computing it again with optimal estimator

In [ ]:
# Build a forest and compute the feature importances
forest = RandomForestClassifier(n_estimators=OptEstimator,random_state=0)
forest.fit(X, y_equidistant)

In [ ]:
# Mean test score of a 250x decision tree 
Forest_scores = cross_validate(forest, X, y_equidistant, cv=cv_strategy)
print('Forest by equidistant interval score {:.3f}'.format(forest.score(X, y_equidistant)))
df_resultat.at[df_resultat.index==4,"method"]="Forest equidistant"
df_resultat.at[df_resultat.index==4,"Cross-validation"]=np.mean(Forest_scores['test_score'])
print('Forest by equidistant interval cross validation {:.3f}'.format(np.mean(Forest_scores['test_score'])))

confusion matrix  with split data

In [ ]:
forest_confusion = RandomForestClassifier(n_estimators=OptEstimator,random_state=0)
forest_confusion.fit(X_tr_equid, y_tr_equid)
confusion_matrix(y_te_equid, forest_confusion.predict(X_te_equid))

In [ ]:
plot_confusion_matrix(forest_confusion, X_te_equid,y_te_equid,cmap=plt.cm.Blues,display_labels=[1,2,3,4,5,6])

In [ ]:
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

In [ ]:
# Print the feature ranking
print("Feature ranking by equidistand interval:")

for f in range(X.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, X.columns[indices[f]], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure(figsize=(20,10))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation = 65)
plt.xlim([-1, X.shape[1]])
plt.show()

---
Random forest - Balanced bins  - Deaths Ratio feature
---

In [ ]:
Estimators=range(100,2000,100)
scoring='accuracy'
cv_val_scores_list = []
cv_val_scores_std = []
cv_val_scores_mean = []
accuracy_scores = []
for Estimator in Estimators:
    forest = RandomForestClassifier(n_estimators=Estimator,random_state=0)
    forest.fit(X, y_balanced)
    cv_val_scores = cross_val_score(forest, X, y_balanced, cv=cv_strategy, scoring=scoring)
    cv_val_scores_list.append(cv_val_scores)
    cv_val_scores_mean.append(cv_val_scores.mean())
    cv_val_scores_std.append(cv_val_scores.std())
    accuracy_scores.append(forest.fit(X, y_balanced).score(X, y_balanced))
    Text="depth %d, cv_val_scores_mean %f  score %f"%(Estimator,cv_val_scores.mean(),forest.score(X, y_balanced))
    print(Text)
cv_val_scores_mean = np.array(cv_val_scores_mean)
cv_val_scores_std = np.array(cv_val_scores_std)
accuracy_scores = np.array(accuracy_scores)

In [ ]:
print("The maximum is by Estimator_balanced = ",(np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1)*100)
OptEstimator=(np.where(cv_val_scores_mean==cv_val_scores_mean.max())[0][0]+1)*100

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,5))
ax.plot(Estimators, cv_val_scores_mean, '-o', label='mean cross-validation accuracy', alpha=0.9)
ax.fill_between(Estimators, cv_val_scores_mean-2*cv_val_scores_std, cv_val_scores_mean+2*cv_val_scores_std, alpha=0.2)
ylim = plt.ylim()
ax.plot(Estimators, accuracy_scores, '-*', label='training_scores', alpha=0.9)
title="Forest accurency - _balanced bins"
ax.set_title(title, fontsize=16)
ax.set_xlabel('Forest Estimator', fontsize=14)
ax.set_ylabel('accuracy', fontsize=14)
ax.set_xticks(Estimators)
ax.legend()

In [ ]:
from sklearn.model_selection import cross_validate
Forest_scores = cross_validate(forest, X, y_balanced, cv=cv_strategy)
df_resultat.at[df_resultat.index==5,"method"]="Forest balanced bins"
df_resultat.at[df_resultat.index==5,"Cross-validation"]=np.mean(Forest_scores['test_score'])
print('Forest by balanced bins of element score {:.3f}'.format(forest.score(X, y_balanced)))
print('Forest by balanced bins of element - mean test {:.3f}'.format(np.mean(Forest_scores['test_score'])))

In [ ]:
forest_confusion = RandomForestClassifier(n_estimators=OptEstimator,random_state=0)
forest_confusion.fit(X_tr_balan, y_tr_balan)
confusion_matrix(y_te_balan, forest_confusion.predict(X_te_balan))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(forest_confusion, X_te_balan, y_te_balan,cmap=plt.cm.Greens,display_labels=[1,2,3,4,5,6])

In [ ]:
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

In [ ]:
# Print the feature ranking
print("Feature ranking by balanced bins:")

for f in range(X.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, X.columns[indices[f]], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure(figsize=(20,10))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation = 65)
plt.xlim([-1, X.shape[1]])
plt.show()

---
Feature reduction to 2 and kneighbour classifier -  - Deaths Ratio feature
---
feature reduction

In [ ]:
pca = PCA(n_components=2)
# Apply PCA
pca.fit(X, y=None); # Unsupervised learning, no y variable
feature_2= pca.transform(X)
feature_2_df=pd.DataFrame(feature_2)
len_cluster, col=feature_2_df.shape

In [ ]:
X_fea_2_tr_equid, X_fea_2_te_equid, y_fea_2_tr_equid, y_fea_2_te_equid = train_test_split(feature_2, y_equidistant, test_size=0.15, random_state=1)
print('Train set equidistant:', X_fea_2_tr_equid.shape, y_fea_2_tr_equid.shape)
print('Testn set equidistant:', X_fea_2_te_equid.shape, y_fea_2_te_equid.shape)
X_fea_2_tr_balan, X_fea_2_te_balan, y_fea_2_tr_balan, y_fea_2_te_balan = train_test_split(feature_2, y_balanced, test_size=0.15, random_state=1)
print('Train set balanced:', X_fea_2_tr_balan.shape, y_fea_2_tr_balan.shape)
print('Testn set balanced:', X_fea_2_te_balan.shape, y_fea_2_te_balan.shape)

---
K neightbour classifier - equidistant bins  - Deaths Ratio feature
---

In [ ]:
neighbors = list(range(1,25))
CV_scores = []

for n_neighborss in neighbors:
    weights='distance'
    feature1=pd.DataFrame(feature_2).loc[:][0]
    feature2=pd.DataFrame(feature_2).loc[:][1]
    h = .02  # step size in the mesh
    # we create an instance of Neighbours Classifier and fit the data.
    # Create a k-NN pipeline
    knn_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('knn', KNeighborsClassifier(n_neighbors=n_neighborss, weights=weights))
    ])
    # Fit estimator
    knn_pipe.fit(feature_2, y_equidistant)
    #
    Z = knn_pipe.predict(feature_2)
    scores = cross_val_score(knn_pipe,feature_2,y_equidistant,cv=cv_strategy )
    CV_scores.append(scores.mean())
    
Missclassification_Error = [1-x for x in CV_scores]
print("the optimal k is ",format(Missclassification_Error.index(min(Missclassification_Error))))
k_Optimal=Missclassification_Error.index(min(Missclassification_Error))
    
from matplotlib.legend_handler import HandlerLine2D
plt.plot( Missclassification_Error)
plt.xlabel("number of neighbors K")
plt.ylabel("Missclassification Error")
plt.show()

In [ ]:
# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF', '#EFFF00', '#B4FCFF', '#FFB4F5'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF', '#FF00F7', '#00FFE6', '#FFA200'])

feature1=pd.DataFrame(feature_2).loc[:][0]
feature2=pd.DataFrame(feature_2).loc[:][1]

weights='distance'
h = .02  # step size in the mesh
# we create an instance of Neighbours Classifier and fit the data.
# Create a k-NN pipeline
knn_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=k_Optimal, weights=weights))
])
# Fit estimator
knn_pipe.fit(feature_2, y_equidistant)
# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
feature1_min, feature1_max = feature1.min() - 1, feature1.max() + 1
feature2_min, feature2_max = feature2.min() - 1, feature2.max() + 1
xx, yy = np.meshgrid(np.arange(feature1_min, feature1_max, h), np.arange(feature2_min, feature2_max, h))
Z = knn_pipe.predict(np.c_[xx.ravel(), yy.ravel()])
Zo = Z.reshape(xx.shape)

In [ ]:
# Put the result into a color plot
plt.figure(figsize=(18, 10))
plt.pcolormesh(xx, yy, Zo, cmap=cmap_light)
#Label1=pd.DataFrame(list(class_index.items()))
Label1=pd.DataFrame([1,2,3,4,5])
# Plot also the training points
plt.scatter(feature1, feature2, c=y_equidistant, cmap=cmap_bold)
cbar = plt.colorbar() 
cbar.ax.set_yticklabels([Level1_equidistant,Level2_equidistant,Level3_equidistant,Level4_equidistant,Level5_equidistant,Level6_equidistant])
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xlabel("feature 1")
plt.ylabel("feature 2")
plt.title("6-Class classification,   equidistant bins -  total infected log10 (k = %i, weights = '%s')" % (k_Optimal, weights))

plt.show()

In [ ]:
knn_scores = cross_validate(knn_pipe, feature_2, y_equidistant, cv=cv_strategy)
df_resultat.at[df_resultat.index==6,"method"]="knn equidistant bins"
df_resultat.at[df_resultat.index==6,"Cross-validation"]=np.mean(knn_scores['test_score'])
print('knn by equidistant bins of element score {:.3f}'.format(knn_pipe.score(feature_2, y_equidistant)))
print('knn by equidistant bins of element - mean test {:.3f}'.format(np.mean(knn_scores['test_score'])))


In [ ]:
knn_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=k_Optimal, weights=weights))
])
# Fit estimator
knn_confusion=knn_pipe.fit(X_fea_2_tr_equid, y_fea_2_tr_equid)
knn_predict = knn_pipe.predict(X_fea_2_te_equid)
confusion_matrix(y_fea_2_te_equid, knn_predict)

In [ ]:
plot_confusion_matrix(knn_confusion, X_fea_2_te_equid, y_fea_2_te_equid,cmap=plt.cm.Blues,display_labels=[1,2,3,4,5,6])

---
K neightbour classifier - balanced bins  - Deaths Ratio feature
---

In [ ]:
neighbors = list(range(1,25))
CV_scores = []

for n_neighborss in neighbors:
    weights='distance'
    feature1=pd.DataFrame(feature_2).loc[:][0]
    feature2=pd.DataFrame(feature_2).loc[:][1]
    h = .02  # step size in the mesh
    # we create an instance of Neighbours Classifier and fit the data.
    # Create a k-NN pipeline
    knn_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('knn', KNeighborsClassifier(n_neighbors=n_neighborss, weights=weights))
    ])
    # Fit estimator
    knn_pipe.fit(feature_2, y_balanced)
    #
    Z = knn_pipe.predict(feature_2)
    scores = cross_val_score(knn_pipe,feature_2,y_balanced,cv=cv_strategy )
    CV_scores.append(scores.mean())
    
Missclassification_Error = [1-x for x in CV_scores]
print("the optimal k is ",format(Missclassification_Error.index(min(Missclassification_Error))))
k_Optimal=Missclassification_Error.index(min(Missclassification_Error))
    
from matplotlib.legend_handler import HandlerLine2D
plt.plot( Missclassification_Error)
plt.xlabel("number of neighbors K")
plt.ylabel("Missclassification Error")
plt.show()

In [ ]:
# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF', '#EFFF00', '#B4FCFF', '#FFB4F5'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF', '#FF00F7', '#00FFE6', '#FFA200'])

feature1=pd.DataFrame(feature_2).loc[:][0]
feature2=pd.DataFrame(feature_2).loc[:][1]

weights='distance'
h = .02  # step size in the mesh
# we create an instance of Neighbours Classifier and fit the data.
# Create a k-NN pipeline
knn_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=k_Optimal, weights=weights))
])
# Fit estimator
knn_pipe.fit(feature_2, y_balanced)
# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
feature1_min, feature1_max = feature1.min() - 1, feature1.max() + 1
feature2_min, feature2_max = feature2.min() - 1, feature2.max() + 1
xx, yy = np.meshgrid(np.arange(feature1_min, feature1_max, h), np.arange(feature2_min, feature2_max, h))
Z = knn_pipe.predict(np.c_[xx.ravel(), yy.ravel()])
Zo = Z.reshape(xx.shape)

In [ ]:
# Put the result into a color plot
plt.figure(figsize=(18, 10))
plt.pcolormesh(xx, yy, Zo, cmap=cmap_light)
#Label1=pd.DataFrame(list(class_index.items()))
Label1=pd.DataFrame([1,2,3,4,5])
# Plot also the training points
plt.scatter(feature1, feature2, c=y_balanced, cmap=cmap_bold)
cbar = plt.colorbar() 
cbar.ax.set_yticklabels([Level1_balanced,Level2_balanced,Level3_balanced,Level4_balanced,Level5_balanced,Level6_balanced])
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xlabel("feature 1")
plt.ylabel("feature 2")
plt.title("6-Class classification,   balanced bins -  total infected log10 (k = %i, weights = '%s')" % (k_Optimal, weights))

plt.show()

In [ ]:

knn_scores = cross_validate(knn_pipe, feature_2, y_balanced, cv=cv_strategy)
df_resultat.at[df_resultat.index==7,"method"]="knn balanced bins"
df_resultat.at[df_resultat.index==7,"Cross-validation"]=np.mean(knn_scores['test_score'])
print('knn by equidistant bins of element score {:.3f}'.format(knn_pipe.score(feature_2, y_balanced)))
print('knn by equidistant bins of element - mean test {:.3f}'.format(np.mean(knn_scores['test_score'])))


In [ ]:
knn_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=k_Optimal, weights=weights))
])
# Fit estimator
knn_confusion=knn_pipe.fit(X_fea_2_tr_balan, y_fea_2_tr_balan)
knn_predict = knn_pipe.predict(X_fea_2_te_balan)

In [ ]:
confusion_matrix(y_fea_2_te_balan, knn_predict)

In [ ]:
plot_confusion_matrix(knn_confusion, X_fea_2_te_balan, y_fea_2_te_balan,cmap=plt.cm.Greens,display_labels=[1,2,3,4,5,6])

Summary  - Deaths Ratio feature
---

In [ ]:
df_resultat

In [ ]:
sns.barplot(x=df_resultat["Cross-validation"],y=df_resultat['method'])
plt.show()

a few surprized that the tree is high by deapth 1, the forest deliver the awaiting factor importance

---
---
PART 3 : daily value trend and prediction
---

we have these tabel
codiv_country_qurantine
codiv_country_Restrictions
codiv_country_without_Restrictions_qurantine
codiv_time_recovered
codiv_time_deaths
codiv_time_confirmed

need to consolidate the country name
group row having same country name

In [ ]:
codiv_time_confirmed

In [ ]:
codiv_time_confirmed_grouped=codiv_time_confirmed.groupby('Country/Region').sum().reset_index()
dfsum=codiv_time_confirmed_grouped.agg(['sum'])
dfsum['Country/Region']="total_confirmed"
codiv_time_confirmed_grouped_sum=codiv_time_confirmed_grouped.append(dfsum)
#
codiv_time_recovered_grouped=codiv_time_recovered.groupby('Country/Region').sum().reset_index()
dfsum=codiv_time_recovered_grouped.agg(['sum'])
dfsum['Country/Region']="total_recovered"
codiv_time_recovered_grouped_sum=codiv_time_recovered_grouped.append(dfsum)
#
codiv_time_deaths_grouped=codiv_time_deaths.groupby('Country/Region').sum().reset_index()
dfsum=codiv_time_deaths_grouped.agg(['sum'])
dfsum['Country/Region']="total_deaths"
codiv_time_deaths_grouped_sum=codiv_time_deaths_grouped.append(dfsum)
codiv_time_deaths_grouped_sum

buiding new table for the 3 cathegory
codiv_country_qurantine
codiv_country_Restrictions
codiv_country_without_Restrictions_qurantine

In [ ]:
#Recovered
Start=1
for countryInd in codiv_country_qurantine['Country']:
    if not codiv_country_short[codiv_country_short['Country']==countryInd].empty:
        if Start==1:
            codiv_country_qurantine_recovered=codiv_time_recovered_grouped_sum[codiv_time_recovered_grouped_sum['Country/Region']==countryInd]
            Start=0
        else:
            codiv_country_qurantine_recovered=codiv_country_qurantine_recovered.append(codiv_time_recovered_grouped_sum[codiv_time_recovered_grouped_sum['Country/Region']==countryInd])
#
Start=1
for countryInd in codiv_country_Restrictions['Country']:
    if not codiv_country_short[codiv_country_short['Country']==countryInd].empty:
        if Start==1:
            codiv_country_Restrictions_recovered=codiv_time_recovered_grouped_sum[codiv_time_recovered_grouped_sum['Country/Region']==countryInd]
            Start=0
        else:
            codiv_country_Restrictions_recovered=codiv_country_Restrictions_recovered.append(codiv_time_recovered_grouped_sum[codiv_time_recovered_grouped_sum['Country/Region']==countryInd])
#
Start=1
for countryInd in codiv_country_without_Restrictions_qurantine['Country']:
    if not codiv_country_short[codiv_country_short['Country']==countryInd].empty:
        if Start==1:
            codiv_country_without_Restrictions_qurantine_recovered=codiv_time_recovered_grouped_sum[codiv_time_recovered_grouped_sum['Country/Region']==countryInd]
            Start=0
        else:
            codiv_country_without_Restrictions_qurantine_recovered=codiv_country_without_Restrictions_qurantine_recovered.append(codiv_time_recovered_grouped_sum[codiv_time_recovered_grouped_sum['Country/Region']==countryInd]) 
#deaths   
Start=1
for countryInd in codiv_country_qurantine['Country']:
    if not codiv_country_short[codiv_country_short['Country']==countryInd].empty:
        if Start==1:
            codiv_country_qurantine_deaths=codiv_time_deaths_grouped_sum[codiv_time_deaths_grouped_sum['Country/Region']==countryInd]
            Start=0
        else:
            codiv_country_qurantine_deaths=codiv_country_qurantine_deaths.append(codiv_time_deaths_grouped_sum[codiv_time_deaths_grouped_sum['Country/Region']==countryInd])
#
Start=1
for countryInd in codiv_country_Restrictions['Country']:
    if not codiv_country_short[codiv_country_short['Country']==countryInd].empty:
        if Start==1:
            codiv_country_Restrictions_deaths=codiv_time_deaths_grouped_sum[codiv_time_deaths_grouped_sum['Country/Region']==countryInd]
            Start=0
        else:
            codiv_country_Restrictions_deaths=codiv_country_Restrictions_deaths.append(codiv_time_deaths_grouped_sum[codiv_time_deaths_grouped_sum['Country/Region']==countryInd])
#
Start=1
for countryInd in codiv_country_without_Restrictions_qurantine['Country']:
    if not codiv_country_short[codiv_country_short['Country']==countryInd].empty:
        if Start==1:
            codiv_country_without_Restrictions_qurantine_deaths=codiv_time_deaths_grouped_sum[codiv_time_deaths_grouped_sum['Country/Region']==countryInd]
            Start=0
        else:
            codiv_country_without_Restrictions_qurantine_deaths=codiv_country_without_Restrictions_qurantine_deaths.append(codiv_time_deaths_grouped_sum[codiv_time_deaths_grouped_sum['Country/Region']==countryInd]) 
#confirmed

Start=1
for countryInd in codiv_country_qurantine['Country']:
    if not codiv_country_short[codiv_country_short['Country']==countryInd].empty:
        if Start==1:
            codiv_country_qurantine_confirmed=codiv_time_confirmed_grouped_sum[codiv_time_confirmed_grouped_sum['Country/Region']==countryInd]
            Start=0
        else:
            codiv_country_qurantine_confirmed=codiv_country_qurantine_confirmed.append(codiv_time_confirmed_grouped_sum[codiv_time_confirmed_grouped_sum['Country/Region']==countryInd])

Start=1
for countryInd in codiv_country_Restrictions['Country']:
    if not codiv_country_short[codiv_country_short['Country']==countryInd].empty:
        if Start==1:
            codiv_country_Restrictions_confirmed=codiv_time_confirmed_grouped_sum[codiv_time_confirmed_grouped_sum['Country/Region']==countryInd]
            Start=0
        else:
            codiv_country_Restrictions_confirmed=codiv_country_Restrictions_confirmed.append(codiv_time_confirmed_grouped_sum[codiv_time_confirmed_grouped_sum['Country/Region']==countryInd])

#codiv_country_Restrictions_confirmed=codiv_country_Restrictions_confirmed.set_index('Country/Region')
#
Start=1
for countryInd in codiv_country_without_Restrictions_qurantine['Country']:
    if not codiv_country_short[codiv_country_short['Country']==countryInd].empty:
        if Start==1:
            codiv_country_without_Restrictions_qurantine_confirmed=codiv_time_confirmed_grouped_sum[codiv_time_confirmed_grouped_sum['Country/Region']==countryInd]
            Start=0
        else:
            codiv_country_without_Restrictions_qurantine_confirmed=codiv_country_without_Restrictions_qurantine_confirmed.append(codiv_time_confirmed_grouped_sum[codiv_time_confirmed_grouped_sum['Country/Region']==countryInd]) 



tabelle available: stats
    codiv_country_without_Restrictions_qurantine_confirmed
    codiv_country_without_Restrictions_qurantine_deaths
    codiv_country_without_Restrictions_qurantine_recovered
    
    codiv_country_qurantine_confirmed
    codiv_country_qurantine_deaths
    codiv_country_qurantine_recovered
    
    codiv_country_Restrictions_confirmed
    codiv_country_Restrictions_deaths
    codiv_country_Restrictions_recovered
    
    
    Build active-infected table

In [ ]:
codiv_country_without_Restrictions_qurantine_active= codiv_country_without_Restrictions_qurantine_confirmed.set_index('Country/Region')-codiv_country_without_Restrictions_qurantine_deaths.set_index('Country/Region')-codiv_country_without_Restrictions_qurantine_recovered.set_index('Country/Region')
codiv_country_qurantine_active= codiv_country_qurantine_confirmed.set_index('Country/Region')-codiv_country_qurantine_deaths.set_index('Country/Region')-codiv_country_qurantine_recovered.set_index('Country/Region')
codiv_country_Restrictions_active= codiv_country_Restrictions_confirmed.set_index('Country/Region')- codiv_country_Restrictions_deaths.set_index('Country/Region')-codiv_country_Restrictions_recovered.set_index('Country/Region')

confirmed Log10
---

In [ ]:
import datetime
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
row, col= codiv_country_qurantine_confirmed.shape
fig, (ax0,ax1,ax2) = plt.subplots(3, sharey=True,figsize=(25,40))
ts = pd.Series(
    np.random.randn(col-1),
    index=pd.date_range('20/1/2020', periods=col-1))
df_qua_conf = pd.DataFrame((np.log1p(codiv_country_qurantine_confirmed.set_index('Country/Region').T).to_numpy()),
    index=ts.index,
    columns=list(codiv_country_qurantine_confirmed.set_index('Country/Region').index))
df_res_conf = pd.DataFrame((np.log1p(codiv_country_Restrictions_confirmed.set_index('Country/Region').T).to_numpy()),
    index=ts.index,
    columns=list(codiv_country_Restrictions_confirmed.set_index('Country/Region').index))
df_whi_conf = pd.DataFrame((np.log1p(codiv_country_without_Restrictions_qurantine_confirmed.set_index('Country/Region').T).to_numpy()),
    index=ts.index,
    columns=list(codiv_country_without_Restrictions_qurantine_confirmed.set_index('Country/Region').index))

ax0.plot(df_qua_conf.index,df_qua_conf)
ax1.plot(df_res_conf.index, df_res_conf)
ax2.plot(df_whi_conf.index, df_whi_conf)
ax0.set_title("quarantine_confirmed")
ax1.set_title("Restrictions_confirmed")
ax2.set_title("without_Restrictions_quarantine_confirmed")
ax0.legend(codiv_country_qurantine_confirmed['Country/Region'].tolist(),loc='upper left')
ax1.legend(codiv_country_Restrictions_confirmed['Country/Region'].tolist(),loc='upper left')
ax2.legend(codiv_country_without_Restrictions_qurantine_confirmed['Country/Region'].tolist(),loc='upper left')
ax0.set_ylabel("quarantine_confirmed")
ax1.set_ylabel("Restrictions_confirmed")
ax2.set_ylabel("without_Restrictions_quarantine_confirmed")

ax0.grid(True)
ax0.xaxis.set_minor_locator(AutoMinorLocator())
ax1.grid(True)
ax1.xaxis.set_minor_locator(AutoMinorLocator())
ax2.grid(True)
ax2.xaxis.set_minor_locator(AutoMinorLocator())
ax0.tick_params(axis="x", rotation=45)
ax1.tick_params(axis="x", rotation=45)
ax2.tick_params(axis="x", rotation=45)
####
for countryindex in codiv_country_qurantine_confirmed['Country/Region']:
    if not codiv_country[codiv_country['Country']==countryindex].empty :
        if codiv_country[codiv_country['Country']==countryindex]['Quarantine'].values[0]!="2000-01-01" and countryindex!="New Zealand":
            QuarantineTime=codiv_country[codiv_country['Country']==countryindex]['Quarantine'].values[0]
            QuarantineTimeD = datetime.datetime.strptime(str(QuarantineTime), "%m/%d/%Y")
            Centervalue=df_qua_conf[df_qua_conf.index==QuarantineTimeD][countryindex].values[0]
            ax0.annotate('Quarantine', (QuarantineTimeD, Centervalue),
                xytext=(0.2, 0.95), textcoords='axes fraction',
                arrowprops=dict(facecolor='red', shrink=0.001),
                fontsize=16,
                horizontalalignment='right', verticalalignment='top')
####
for countryindex in codiv_country_Restrictions_confirmed['Country/Region']:
    if not codiv_country[codiv_country['Country']==countryindex].empty :
        if codiv_country[codiv_country['Country']==countryindex]['Restrictions'].values[0]!="2000-01-01":
            RestrictionsTime=codiv_country[codiv_country['Country']==countryindex]['Restrictions'].values[0]
            RestrictionsTimeD = datetime.datetime.strptime(str(RestrictionsTime), "%m/%d/%Y")
            Centervalue=df_res_conf[df_res_conf.index==RestrictionsTimeD][countryindex].values[0]
            ax1.annotate('Restrictions', (RestrictionsTimeD, Centervalue),
                xytext=(0.2, 0.95), textcoords='axes fraction',
                arrowprops=dict(facecolor='red', shrink=0.001),
                fontsize=16,
                horizontalalignment='right', verticalalignment='top')

The countries without restrictions and quarantine definitely have an increasing epidemic.

The countries with restrictions show a slowdown in development.

The countries with the quarantine have been successful in achieving a better result.

Quarantine and restrictions are political decisions. How they are used differs from country to country, also how violations of the law are punished or monitor is dependent on country

deaths Log10
---

In [ ]:
import datetime
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
row, col= codiv_country_qurantine_deaths.shape
fig, (ax0,ax1,ax2) = plt.subplots(3, sharey=True,figsize=(25,40))
ts = pd.Series(
    np.random.randn(col-1),
    index=pd.date_range('20/1/2020', periods=col-1))
df_qua_conf = pd.DataFrame((np.log1p(codiv_country_qurantine_deaths.set_index('Country/Region').T).to_numpy()),
    index=ts.index,
    columns=list(codiv_country_qurantine_deaths.set_index('Country/Region').index))
df_res_conf = pd.DataFrame((np.log1p(codiv_country_Restrictions_deaths.set_index('Country/Region').T).to_numpy()),
    index=ts.index,
    columns=list(codiv_country_Restrictions_deaths.set_index('Country/Region').index))
df_whi_conf = pd.DataFrame((np.log1p(codiv_country_without_Restrictions_qurantine_deaths.set_index('Country/Region').T).to_numpy()),
    index=ts.index,
    columns=list(codiv_country_without_Restrictions_qurantine_deaths.set_index('Country/Region').index))

ax0.plot(df_qua_conf.index,df_qua_conf)
ax1.plot(df_res_conf.index, df_res_conf)
ax2.plot(df_whi_conf.index, df_whi_conf)
ax0.set_title("quarantine_deaths")
ax1.set_title("Restrictions_deaths")
ax2.set_title("without_Restrictions_quarantine_deaths")
ax0.legend(codiv_country_qurantine_deaths['Country/Region'].tolist(),loc='upper left')
ax1.legend(codiv_country_Restrictions_deaths['Country/Region'].tolist(),loc='upper left')
ax2.legend(codiv_country_without_Restrictions_qurantine_deaths['Country/Region'].tolist(),loc='upper left')
ax0.set_ylabel("quarantine_deaths")
ax1.set_ylabel("Restrictions_deaths")
ax2.set_ylabel("without_Restrictions_quarantine_deaths")

ax0.grid(True)
ax0.xaxis.set_minor_locator(AutoMinorLocator())
ax1.grid(True)
ax1.xaxis.set_minor_locator(AutoMinorLocator())
ax2.grid(True)
ax2.xaxis.set_minor_locator(AutoMinorLocator())
ax0.tick_params(axis="x", rotation=45)
ax1.tick_params(axis="x", rotation=45)
ax2.tick_params(axis="x", rotation=45)
####
for countryindex in codiv_country_qurantine_deaths['Country/Region']:
    if not codiv_country[codiv_country['Country']==countryindex].empty :
        if codiv_country[codiv_country['Country']==countryindex]['Quarantine'].values[0]!="2000-01-01" and countryindex!="New Zealand":
            QuarantineTime=codiv_country[codiv_country['Country']==countryindex]['Quarantine'].values[0]
            QuarantineTimeD = datetime.datetime.strptime(str(QuarantineTime), "%m/%d/%Y")
            Centervalue=df_qua_conf[df_qua_conf.index==QuarantineTimeD][countryindex].values[0]
            ax0.annotate('Quarantine', (QuarantineTimeD, Centervalue),
                xytext=(0.2, 0.95), textcoords='axes fraction',
                arrowprops=dict(facecolor='red', shrink=0.001),
                fontsize=16,
                horizontalalignment='right', verticalalignment='top')
####
for countryindex in codiv_country_Restrictions_deaths['Country/Region']:
    if not codiv_country[codiv_country['Country']==countryindex].empty :
        if codiv_country[codiv_country['Country']==countryindex]['Restrictions'].values[0]!="2000-01-01":
            RestrictionsTime=codiv_country[codiv_country['Country']==countryindex]['Restrictions'].values[0]
            RestrictionsTimeD = datetime.datetime.strptime(str(RestrictionsTime), "%m/%d/%Y")
            Centervalue=df_res_conf[df_res_conf.index==RestrictionsTimeD][countryindex].values[0]
            ax1.annotate('Restrictions', (RestrictionsTimeD, Centervalue),
                xytext=(0.2, 0.95), textcoords='axes fraction',
                arrowprops=dict(facecolor='red', shrink=0.001),
                fontsize=16,
                horizontalalignment='right', verticalalignment='top')

Till now are 4-5 main virus variant

in china was 2 version (A) , one agressiv in wuhan, an other one less in the rest of the country (B).

In rest of asia was also an other mutation

in france, spain, italy is also an different the number of deaths is different then in germany/and other eastern countries.
In france was discovered the first virus  in december

that all means it is not possible on the graphic to learn something due of these mutations.

active Log10
---

In [ ]:
import datetime
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
row, col= codiv_country_qurantine_active.shape
fig, (ax0,ax1,ax2) = plt.subplots(3, sharey=True,figsize=(25,40))
ts = pd.Series(
    np.random.randn(col),
    index=pd.date_range('20/1/2020', periods=col))
df_qua_conf = pd.DataFrame((np.log1p(codiv_country_qurantine_active.T).to_numpy()),
    index=ts.index,
    columns=list(codiv_country_qurantine_active.index))
df_res_conf = pd.DataFrame((np.log1p(codiv_country_Restrictions_active.T).to_numpy()),
    index=ts.index,
    columns=list(codiv_country_Restrictions_active.index))
df_whi_conf = pd.DataFrame((np.log1p(codiv_country_without_Restrictions_qurantine_active.T).to_numpy()),
    index=ts.index,
    columns=list(codiv_country_without_Restrictions_qurantine_active.index))

ax0.plot(df_qua_conf.index,df_qua_conf)
ax1.plot(df_res_conf.index, df_res_conf)
ax2.plot(df_whi_conf.index, df_whi_conf)
ax0.set_title("quarantine_active")
ax1.set_title("Restrictions_active")
ax2.set_title("without_Restrictions_quarantine_active")
ax0.legend(codiv_country_qurantine_deaths['Country/Region'].tolist(),loc='upper left')
ax1.legend(codiv_country_Restrictions_deaths['Country/Region'].tolist(),loc='upper left')
ax2.legend(codiv_country_without_Restrictions_qurantine_deaths['Country/Region'].tolist(),loc='upper left')
ax0.set_ylabel("quarantine_active")
ax1.set_ylabel("Restrictions_active")
ax2.set_ylabel("without_Restrictions_qurantine_active")

ax0.grid(True)
ax0.xaxis.set_minor_locator(AutoMinorLocator())
ax1.grid(True)
ax1.xaxis.set_minor_locator(AutoMinorLocator())
ax2.grid(True)
ax2.xaxis.set_minor_locator(AutoMinorLocator())
ax0.tick_params(axis="x", rotation=45)
ax1.tick_params(axis="x", rotation=45)
ax2.tick_params(axis="x", rotation=45)
####
for countryindex in codiv_country_qurantine_deaths['Country/Region']:
    if not codiv_country[codiv_country['Country']==countryindex].empty :
        if codiv_country[codiv_country['Country']==countryindex]['Quarantine'].values[0]!="2000-01-01" and countryindex!="New Zealand":
            QuarantineTime=codiv_country[codiv_country['Country']==countryindex]['Quarantine'].values[0]
            QuarantineTimeD = datetime.datetime.strptime(str(QuarantineTime), "%m/%d/%Y")
            Centervalue=df_qua_conf[df_qua_conf.index==QuarantineTimeD][countryindex].values[0]
            ax0.annotate('Quarantine', (QuarantineTimeD, Centervalue),
                xytext=(0.2, 0.95), textcoords='axes fraction',
                arrowprops=dict(facecolor='red', shrink=0.001),
                fontsize=16,
                horizontalalignment='right', verticalalignment='top')
####
for countryindex in codiv_country_Restrictions_deaths['Country/Region']:
    if not codiv_country[codiv_country['Country']==countryindex].empty :
        if codiv_country[codiv_country['Country']==countryindex]['Restrictions'].values[0]!="2000-01-01":
            RestrictionsTime=codiv_country[codiv_country['Country']==countryindex]['Restrictions'].values[0]
            RestrictionsTimeD = datetime.datetime.strptime(str(RestrictionsTime), "%m/%d/%Y")
            Centervalue=df_res_conf[df_res_conf.index==RestrictionsTimeD][countryindex].values[0]
            ax1.annotate('Restrictions', (RestrictionsTimeD, Centervalue),
                xytext=(0.2, 0.95), textcoords='axes fraction',
                arrowprops=dict(facecolor='red', shrink=0.001),
                fontsize=16,
                horizontalalignment='right', verticalalignment='top')

---
Part 4 : Fitting the data to a model and predictions
---

explanations:
---
I dont target the total number of infected, but only the active number of infected. 
The goal is to predict when we will arrive under a low level of infected.
Only active infected represent a danger, not the recovered persone.
```
```
first i will fit the China active  courve to a model
The epidemic curve is a standard epidemical curve.
After i will transpose theis epidemic model to the other countries.
For the transposition it will be 3 fitting parameter : 1. start point 2. The high point 3. The large 
The Model can not absorbe the suddently adjustment of measuring method of a country, but global it fitt very well

In [ ]:
from IPython.display import Image
Image(filename=os.path.join('.//', 'shift.JPG'))

The issue highlighted in this article [Forecasting s-curves is hard](https://constancecrozier.com/2020/04/16/forecasting-s-curves-is-hard/) is that forecasting the numbers isn't really accurate at the beginning of the cruve until it starts to decrease. For this reason, the models below only provide good insights from China as a model after the epidemic "peak".

Also, the "S curve" model (or Normal curve) only works to model for a single epidemic event which might not be the case due to change in policy ex. end of quarantine or for some countries where the data was changed. The data is not clean in the sense that some countries such as Germany stopped recording during weekends. As another example, the policy can suddenly change as in Wuhan where it was decided to systematically test the population (15th May).

I decided to use a polymial curve to highlight those sudden changes even if the model will only work within the range of data and shouldn't be use to forecast future due to boundary effects as explained here: https://twitter.com/laurabronner/status/1257743918919151616 - the curves in the plots below are limited to the start/end of data points I have.

In [ ]:
# evalue the stadt date of the stats
StartDate=codiv_country_Restrictions_active.T.index[0]
print(" Epidemy StartDate = ",StartDate)
column_names = ["date"]
dfdate = pd.DataFrame(columns = column_names)
for Dateindex in range(20):
    df_date1 = pd.DataFrame ([[str(pd.Timestamp(StartDate,unit="D")+ pd.Timedelta(days=Dateindex))]], columns = ["date"])
    dfdate=dfdate.append(df_date1, ignore_index = True)


In [ ]:
# Root mean squared error (RMSE)
def rmse(y, y_pred):
    return np.sqrt(np.mean(np.square(y - y_pred)))

In [ ]:
import scipy.stats as stats
import datetime
def Polifq(Country,Version="restrictions",shift=0,trigger=40):
    # Version= works for restrictions, quarantin and without_Restrictions_qurantine.
    if Version=="restrictions":
        df_codiv=codiv_country_Restrictions_active.T[Country]
    else:
        if Version=="quarantine":
            df_codiv=codiv_country_qurantine_active.T[Country]
        else:
            df_codiv=codiv_country_without_Restrictions_qurantine_active.T[Country]
    #######################################
    ######################## parameter setup
    StartCountryIndex=0
    ShiftIndex=0
    x_tr1_China_model=0
    y_tr1_China_model=0
    highfactor=1.0
    LargeFactor=1.0
    ##############################################################################
    ####################################### fitting the Country ##################
    ######################## filter outlier for the Country
    filter0 = np.abs(df_codiv - df_codiv.mean()) > (3 * df_codiv.std())
    outliers = df_codiv.loc[filter0]
    df_codiv = df_codiv.drop(outliers.index, axis=0)
    ###########
    # max actual value of the country
    maxCountry=max(df_codiv)
    #######################################
    ########### Build the dataframe for the country
    df_country = pd.DataFrame(columns = [Country,"Value","time"], dtype='int')
    for Index in range(0,len(df_codiv)):
        df_codiv1 = pd.DataFrame ([[int(df_codiv[Index]*highfactor),int((Index)*LargeFactor)+ShiftIndex,str(pd.Timestamp(df_codiv.index[Index])+ pd.Timedelta(days=ShiftIndex))]], columns = [Country,"Value","time"])
        df_country=df_country.append(df_codiv1, ignore_index = True)
    #######################################
    #################### polyfit 10gr for the country value
    x_tr1_Country=df_country["Value"]
    y_tr1_Country=df_country[Country]
    #
    # * Polyfit with degree 10
    coefs_Country_poly10 = np.polyfit(x_tr1_Country, y_tr1_Country, deg=10) # Fit to train data
    ##############################################################################
    ##############################################################################
    coefs_predict_poly10=0
    #######################################
    ############### Now shifting the china model to fitt to the country
    if shift==1:
        Start=1
        ####### for the country
        maxChina=max(codiv_country_qurantine_active.T["China"])
        for Index in range(0,len(df_codiv)):
            #look for the start position of the country epidemy, it should be higher then the trigger value, it will be setup only by start of procedure ( STart=1)
            if df_codiv[Index]>trigger*(maxCountry*1.0/maxChina) and Start==1 and shift==1:
                
                if maxCountry>maxChina:
                    trigger=trigger*(maxCountry*1.5/maxChina)
                StartCountryIndex=Index
                Start=0
            # register the index of the coutry max position
            if df_codiv[Index]==maxCountry:
                maxCountryIndex=Index
        # deliver the actual distance between start and maximum for the country
        LargeCountry=maxCountryIndex-StartCountryIndex
        #################################
        #######################################
        ####### for china
        df_codiv_China_ref=codiv_country_qurantine_active.T["China"]
        #######################################
        ###### outiler
        filter0 = np.abs(df_codiv_China_ref - df_codiv_China_ref.mean()) > (3 * df_codiv_China_ref.std())
        outliers = df_codiv_China_ref.loc[filter0]
        df_codiv = df_codiv_China_ref.drop(outliers.index, axis=0)
        #######################################
        ###### check for the start point by 40 infected
        maxChina=max(df_codiv_China_ref)
        Start=1
        for Index in range(0,len(df_codiv_China_ref)):
            if df_codiv_China_ref[Index]>60 and Start==1 and shift==1:
                StartChinaIndex=Index
                Start=0
            if df_codiv_China_ref[Index]==maxChina:
                maxChinaIndex=Index
        #######################################
        ###### start value for the fitting
        highfactor=maxCountry*1.0/maxChina
        LargeChina=maxChinaIndex-StartChinaIndex
        LargeFactor=LargeCountry*1.0/LargeChina
        ShiftIndex=StartCountryIndex-StartChinaIndex
        ##########OPTIMISATION 
        step=0.02
        ############# the target
        # Predictions with the current a,b values
        ################# generate the function country is china
        df_country = pd.DataFrame(columns = ["China","Value","time"], dtype='int')
        for Index in range(0,len(df_codiv)):
            df_codiv1 = pd.DataFrame ([[int(df_codiv[Index]*highfactor),int((Index)*LargeFactor)+ShiftIndex,str(pd.Timestamp(df_codiv.index[Index])+ pd.Timedelta(days=ShiftIndex))]], columns = ["China","Value","time"])
            df_country=df_country.append(df_codiv1, ignore_index = True)
        x_tr1_China_model=df_country["Value"]
        y_tr1_China_model=df_country["China"]      
        error0 = rmse(y_tr1_China_model,y_tr1_Country)
        error1=0
        for runNr in range(1,60,1):
            if error1<error0:
                LargeFactor=LargeFactor+step
                error0=error1
                ################# generate the function
                df_country = pd.DataFrame(columns = ["China","Value","time"], dtype='int')
                for Index in range(0,len(df_codiv)):
                    df_codiv1 = pd.DataFrame ([[int(df_codiv[Index]*highfactor),int((Index)*LargeFactor)+ShiftIndex,str(pd.Timestamp(df_codiv.index[Index])+ pd.Timedelta(days=ShiftIndex))]], columns = ["China","Value","time"])
                    df_country=df_country.append(df_codiv1, ignore_index = True)
                x_tr1_China_model=df_country["Value"]
                y_tr1_China_model=df_country["China"] 
                error1 = rmse(y_tr1_China_model,y_tr1_Country)
            if error1<error0:
                LargeFactor=LargeFactor+step
                error0=error1
                ################# generate the function
                df_country = pd.DataFrame(columns = ["China","Value","time"], dtype='int')
                for Index in range(0,len(df_codiv)):
                    df_codiv1 = pd.DataFrame ([[int(df_codiv[Index]*highfactor),int((Index)*LargeFactor)+ShiftIndex,str(pd.Timestamp(df_codiv.index[Index])+ pd.Timedelta(days=ShiftIndex))]], columns = ["China","Value","time"])
                    df_country=df_country.append(df_codiv1, ignore_index = True)
                x_tr1_China_model=df_country["Value"]
                y_tr1_China_model=df_country["China"] 
                error1 = rmse(y_tr1_China_model,y_tr1_Country)
            ##############
            if error1<error0:
                highfactor=highfactor+step
                error0=error1
                ################# generate the function
                df_country = pd.DataFrame(columns = ["China","Value","time"], dtype='int')
                for Index in range(0,len(df_codiv)):
                    df_codiv1 = pd.DataFrame ([[int(df_codiv[Index]*highfactor),int((Index)*LargeFactor)+ShiftIndex,str(pd.Timestamp(df_codiv.index[Index])+ pd.Timedelta(days=ShiftIndex))]], columns = ["China","Value","time"])
                    df_country=df_country.append(df_codiv1, ignore_index = True)
                x_tr1_China_model=df_country["Value"]
                y_tr1_China_model=df_country["China"] 
                error1 = rmse(y_tr1_China_model,y_tr1_Country)
            #################
            if error1<error0:
                highfactor=highfactor+step
                error0=error1
                ################# generate the function
                df_country = pd.DataFrame(columns = ["China","Value","time"], dtype='int')
                for Index in range(0,len(df_codiv)):
                    df_codiv1 = pd.DataFrame ([[int(df_codiv[Index]*highfactor),int((Index)*LargeFactor)+ShiftIndex,str(pd.Timestamp(df_codiv.index[Index])+ pd.Timedelta(days=ShiftIndex))]], columns = ["China","Value","time"])
                    df_country=df_country.append(df_codiv1, ignore_index = True)
                x_tr1_China_model=df_country["Value"]
                y_tr1_China_model=df_country["China"] 
                error1 = rmse(y_tr1_China_model,y_tr1_Country)
        #after having this fitting, we can fit the modified china data  on a 10grade polynome
        #
        # * Polyfit with degree 10
        coefs_predict_poly10 = np.polyfit(x_tr1_China_model, y_tr1_China_model, deg=10) # Fit to train data
    return coefs_Country_poly10,x_tr1_Country,y_tr1_Country,coefs_predict_poly10,x_tr1_China_model,y_tr1_China_model,StartCountryIndex

In [ ]:
# how much data are available for a country
number,lendata=codiv_country_Restrictions_active.shape
x_values = np.linspace(0, lendata+40, num=lendata+30)

In [ ]:
#compute the ymax china
coefs_China,x_tr_China,y_tr_China,coefs_China_prev,x_tr_China_prev,y_tr_China_prev,StartCountryIndex=Polifq("China",shift=0,Version="quarantine",trigger=70)
lendataChina=max(x_tr_China)-7
y_China = np.polyval(coefs_China, x_values)
ymaxchina=int(max(y_tr_China)*1.5)

In [ ]:
StartDate=codiv_country_Restrictions_active.T.index[0]
column_names = ["date"]
dfdate = pd.DataFrame(columns = column_names)
for Dateindex in range(0,lendata*20,10):
    df_date1 = pd.DataFrame ([[str(pd.Timestamp(StartDate,unit="D")+ pd.Timedelta(days=Dateindex))]], columns = ["date"])
    dfdate=dfdate.append(df_date1, ignore_index = True)

Quarantine
---

In [ ]:
for countryindex in codiv_country_qurantine_active.T:
#codiv_country_quarantine_active.T:
    if countryindex=="China":
        coefs_Country,x_tr_Country,y_tr_Country,coefs_Country_prev,x_tr_Country_prev,y_tr_Country_prev,StartCountryIndex=Polifq(countryindex,shift=0,Version="quarantine",trigger=70)
        y_Country = np.polyval(coefs_Country, x_values)
        ymax=0
    else:
        coefs_Country,x_tr_Country,y_tr_Country,coefs_Country_prev,x_tr_Country_prev,y_tr_Country_prev,StartCountryIndex=Polifq(countryindex,shift=1,Version="quarantine",trigger=170)
        y_Country = np.polyval(coefs_Country, x_values)
        y_Country_prev = np.polyval(coefs_Country_prev, x_values)
        #adjust the ymax of the graphic
        ymax=int(max(y_Country_prev[StartCountryIndex:lendataChina])*1.40)
    if ymaxchina>ymax:
        ymax=ymaxchina
    #
    fig = plt.figure(figsize= (18, 10))
    #china reference country
    plt.scatter(x_tr_China, y_tr_China, s=10)
    plt.plot(x_values[0:lendataChina], y_China[0:lendataChina], label='China',c="blue")
    #plot the second country
    if countryindex!="China":
        plt.scatter(x_tr_Country, y_tr_Country, s=10)
        plt.plot(x_values[0:lendataChina], y_Country[0:lendataChina], label=countryindex,c="magenta")
        plt.scatter(x_tr_Country_prev,y_tr_Country_prev, s=10,c="black")
        PredictionName='%s-prediction'%(countryindex,)
        plt.plot(x_values, y_Country_prev, label=PredictionName,c="brown",ls='dashed')
    plt.ylim(-5,ymax)
    plt.ylabel("Number")
    plt.grid(True)
    tickvalues = range(0,len(dfdate),10)
    plt.xticks(ticks = tickvalues ,labels = dfdate["date"], rotation = 75)
    if countryindex!="China":
        Title='%s-prediction - Quarantine'%(countryindex,)
        plt.title(Title)
    else:
        plt.title("China")
    Texte=""
    if countryindex=="Belgium":
        Texte="Belgium only success to slow the infection. The model wait for the peak to fit better"
    if countryindex=="France":
        Texte="France was relaxing the quarantine? the model need the next days data to fit better. the datas are not clear"      
    if countryindex=="Germany" or countryindex=="Austria":
        Texte="Germany was relaxing the quarantine. but the model fit mostly good"      
    if countryindex=="India" or countryindex=="Argentina" or countryindex=="Peru" or countryindex=="Colombia":
        Texte=" The model wait for the peak to fit better"       
    if countryindex=="Spain":
        Texte=" Spain had no clear data, the model need the next days data to fit better"  
    if countryindex=="China":
        Texte=" China is the reference country, but we see they have to face to some epidemic restart ( imported case)."  
    plt.text(1, ymax-10000, Texte, fontsize=15)
    plt.legend()
    plt.show()

Restrictions
---

In [ ]:
number,lendata=codiv_country_Restrictions_active.shape
x_values = np.linspace(0, lendata+40, num=lendata+30)
coefs_China,x_tr_China,y_tr_China,coefs_China_prev,x_tr_China_prev,y_tr_China_prev,StartCountryIndex=Polifq("China",shift=0,Version="quarantine",trigger=70)
lendataChina=max(x_tr_China)-7
y_China = np.polyval(coefs_China, x_values)
ymaxchina=int(max(y_tr_China)*1.5)

In [ ]:
for countryindex in codiv_country_Restrictions_active.T:
#codiv_country_quarantine_active.T:
    # China is only in quarantine group
    if countryindex=="Japan":
        trigger=4500
    else:
        trigger=250
    coefs_Country,x_tr_Country,y_tr_Country,coefs_Country_prev,x_tr_Country_prev,y_tr_Country_prev,StartCountryIndex=Polifq(countryindex,shift=1,Version="restrictions",trigger=trigger)
    y_Country = np.polyval(coefs_Country, x_values)
    y_Country_prev = np.polyval(coefs_Country_prev, x_values)
    ymax=int(max(y_Country_prev[StartCountryIndex:lendataChina])*1.40)
    #adjust the ymax limit
    if ymaxchina>ymax:
        ymax=ymaxchina
    #
    fig = plt.figure(figsize= (18, 10))
    plt.scatter(x_tr_China, y_tr_China, s=10)
    plt.plot(x_values[0:lendataChina], y_China[0:lendataChina], label='China',c="blue")
    if countryindex!="China":
        plt.scatter(x_tr_Country, y_tr_Country, s=10)
        plt.plot(x_values[0:lendataChina], y_Country[0:lendataChina], label=countryindex,c="magenta")
        plt.scatter(x_tr_Country_prev,y_tr_Country_prev, s=10,c="black")
        PredictionName='%s-prediction'%(countryindex,)
        plt.plot(x_values, y_Country_prev, label=PredictionName,c="brown",ls='dashed')
    plt.ylim(-5,ymax)
    plt.ylabel("Number")
    plt.grid(True)
    tickvalues = range(0,len(dfdate),10)
    plt.xticks(ticks = tickvalues ,labels = dfdate["date"], rotation = 75)
    if countryindex!="China":
        Title='%s-prediction - Restrictions'%(countryindex,)
        plt.title(Title)
    else:
        plt.title("China")
    ##
    Texte=""
    if countryindex=="Ireland":
        Texte="Ireland had no clear data, the model need the next days data to fit better. Are the information trustfull?"
    if countryindex=="Japan":
        Texte="Japan tried long time to contain the epidemy, after it gone up. the start threshold should be higher" 
    if countryindex=="Canada" or countryindex=="Netherlands" or countryindex=="Peru" or countryindex=="Colombia"or countryindex=="Portugal"or countryindex=="Sweden"or countryindex=="United Kingdom":
        Texte=" The model wait for the peak to fit better"     
    if countryindex=="Norway" or countryindex=="Poland":
        Texte=" The model wait for the peak to fit better,they take long time to reach it" 
    plt.text(1, ymax-10000, Texte, fontsize=15)
    ##
    plt.legend()
    plt.show()

No Quarantine  and  No Restrictions
---

In [ ]:
number,lendata=codiv_country_Restrictions_active.shape
x_values = np.linspace(0, lendata+40, num=lendata+30)
coefs_China,x_tr_China,y_tr_China,coefs_China_prev,x_tr_China_prev,y_tr_China_prev,StartCountryIndex=Polifq("China",shift=0,Version="quarantine",trigger=70)
lendataChina=max(x_tr_China)-7
y_China = np.polyval(coefs_China, x_values)
ymaxchina=int(max(y_tr_China)*1.5)

In [ ]:
for countryindex in codiv_country_without_Restrictions_qurantine_active.T:
#codiv_country_quarantine_active.T:
    # china is only in quarntine group
    if countryindex=="Singapore" or countryindex=="Qatar" or countryindex=="Kuwait":
        trigger=2000
    else:
        trigger=300
    coefs_Country,x_tr_Country,y_tr_Country,coefs_Country_prev,x_tr_Country_prev,y_tr_Country_prev,StartCountryIndex=Polifq(countryindex,shift=1,Version="NoQuaNoRES",trigger=trigger)
    y_Country = np.polyval(coefs_Country, x_values)
    y_Country_prev = np.polyval(coefs_Country_prev, x_values)
    # adjust the ymax graph limit
    if countryindex=="US":
        ymax=1500000
    else:
        if countryindex=="Brazil":
            ymax=180000
        else:
            ymax=60000
    fig = plt.figure(figsize= (18, 10))
    plt.scatter(x_tr_China, y_tr_China, s=10)
    plt.plot(x_values[0:lendataChina], y_China[0:lendataChina], label='China',c="blue")
    if countryindex!="China":
        plt.scatter(x_tr_Country, y_tr_Country, s=10)
        plt.plot(x_values[0:lendataChina], y_Country[0:lendataChina], label=countryindex,c="magenta")
        plt.scatter(x_tr_Country_prev,y_tr_Country_prev, s=10,c="black")
        PredictionName='%s-prediction'%(countryindex,)
        plt.plot(x_values, y_Country_prev, label=PredictionName,c="brown",ls='dashed')
    plt.ylim(-5,ymax)
    plt.ylabel("Number")
    plt.grid(True)
    tickvalues = range(0,len(dfdate),10)
    plt.xticks(ticks = tickvalues ,labels = dfdate["date"], rotation = 75)
    if countryindex!="China":
        Title='%s-prediction - No-Restrictions No-quarantine'%(countryindex,)
        plt.title(Title)
    else:
        plt.title("China")
    ##
    Texte=""
    if countryindex=="Armenia" or countryindex=="Bahrain" or countryindex=="Bangladesh" or countryindex=="Belarus"or countryindex=="Brazil"or countryindex=="Chile"or countryindex=="United Kingdom"or countryindex=="US":
        Texte=" The model wait for the peak to fit better"  
    if countryindex=="Ecuador" or countryindex=="Indonesia" or countryindex=="Kuwait" or countryindex=="Mexico" or countryindex=="Oman" or countryindex=="Pakistan" or countryindex=="Quatar" or countryindex=="South Africa":
        Texte=" The model wait for the peak to fit better"
    if countryindex=="Dominican Republic" or countryindex=="Ghana":
        Texte=" The model wait for more data to fit better"
    plt.text(1, ymax-10000, Texte, fontsize=15)
    plt.legend()
    plt.show()